# Model Comparisons

Take the learnings and use 500 seq_len and 16 batch_size

1. LSTM Standard
2. RNN
3. CNN
4. GRU
5. TCN
5. ConvLSTM
6. FFN

In [ ]:
!pip install neurodsp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
from neurodsp.burst import detect_bursts_dual_threshold

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix, f1_score, classification_report, matthews_corrcoef, recall_score
from sklearn.model_selection import train_test_split, KFold
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
import sys

import json
from pprint import pprint

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, accuracy_score


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional


!pip install keras-tcn
from tcn import TCN, tcn_full_summary


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Get the current working directory (where your notebook is)
current_directory = os.getcwd()


# Construct the full path to your data_utils.py file
data_utils_path = '/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Thesis Files/data_utils.py'  # Replace with the actual path

# Add the directory containing data_utils.py to the Python path
sys.path.append(os.path.dirname(data_utils_path))  # Add parent directory of data_utils.py
# Now you can import the custom module
import data_utils as du
import analysis_utils as au

sys.path = current_directory
print(sys.path)

Mounted at /content/drive
/content


## Model Set-Up

### Model Definitions

In [ ]:
from scipy.signal import hilbert
from scipy.signal import butter, filtfilt

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    """
    Applies a Butterworth bandpass filter to the signal data.

    Args:
        data (np.ndarray): The input signal.
        lowcut (float): Lower cutoff frequency in Hz.
        highcut (float): Upper cutoff frequency in Hz.
        fs (float): Sampling frequency in Hz.
        order (int, optional): Order of the filter. Defaults to 4.

    Returns:
        np.ndarray: The filtered signal.
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    # Applies a butter bandpass filter
    b, a = butter(order, [low, high], btype='band')
    # Applies digital filter to minimize distortion introduced by the filters.
    y = filtfilt(b, a, data)
    return y

# Step 1: Compute the Hilbert transform amplitude for the signal
def compute_hilbert_features(X, fs=250):  # Accepts 3D input X
    """
    Computes Hilbert amplitude features with frequency-dependent filter orders and smoothing.
    Flattens the input, computes amplitudes, and reshapes back to the original shape.

    Args:
        X (np.ndarray): The input signal data (3D array).
        fs (int, optional): Sampling frequency. Defaults to 250.

    Returns:
        np.ndarray: The Hilbert amplitude features (3D array with the same shape as input X).
    """
    original_shape = X.shape

    # Flatten the input data
    signal_flat = X.flatten()

    # Define frequency bands with filter orders and smoothing factors
    frequency_bands = {
        'theta': (4, 7, 2, 0.0),  # (lowcut, highcut, filter_order, smoothing_factor)
        'alpha': (8, 11, 2, 0.0),
        'beta': (12, 28, 6, 0.3),
        'gamma': (30, 100, 8, 0.4)
    }

    hilbert_features = []

    for band_name, (lowcut, highcut, filter_order, smoothing_factor) in frequency_bands.items():
        # Apply bandpass filter with specified order
        filtered_signal = butter_bandpass_filter(signal_flat, lowcut, highcut, fs, order=filter_order)

        # Compute Hilbert amplitude envelope
        amplitude_envelope = np.abs(hilbert(filtered_signal))

        # Apply smoothing (e.g., moving average) only if smoothing_factor > 0
        if smoothing_factor > 0:
            window_size = int(smoothing_factor * fs)  # Adjust window size based on frequency and smoothing factor
            amplitude_envelope = np.convolve(amplitude_envelope, np.ones(window_size) / window_size, mode='same')
        print(band_name, amplitude_envelope.shape)
        hilbert_features.append(amplitude_envelope)

    # Reshape the features back to the original 3D shape
    hilbert_features_3d = np.column_stack(hilbert_features).reshape(original_shape + (4,))  # Add feature dimension
    print(hilbert_features_3d.shape)
    return hilbert_features_3d

In [ ]:
# Re-import necessary modules after execution state reset
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Conv1D, ConvLSTM1D, Reshape, RNN, GRU, SimpleRNN, TimeDistributed, ConvLSTM2D, GlobalAveragePooling1D


# Redefine the models
def create_lstm_baseline(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(32, return_sequences=True),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def create_bidirectional_lstm_balanced(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(22, return_sequences=True)),  # 24 units per direction
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# Needs Sliding windows as well.
def create_cnn_balanced(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters=40, kernel_size=3, activation='relu'),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# CNN Per timestep.
def create_cnn_per_timestep(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters=40, kernel_size=3, activation='relu', padding='same'),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        TimeDistributed(Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output"))
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def create_gru_balanced(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        GRU(37, return_sequences=True),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def create_rnn_balanced(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        SimpleRNN(67, return_sequences=True),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def create_tcn_balanced(input_shape, binary, kernel_size=3):
    model = Sequential([
        Input(shape=input_shape),
        TCN(kernel_size=kernel_size, nb_filters=15, return_sequences=True, dilations=[1, 2, 4, 8]),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 1D ConvLSTM needs sliding windows
def create_convLSTM_model(input_shape, binary):
    # Create ConvLSTM model
    model = Sequential([
        Input(shape=(input_shape[0],1,1,input_shape[1])),
        ConvLSTM2D(filters=32, kernel_size=(1, 1), activation='relu', return_sequences=True),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])

    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def create_ffn_balanced(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Dense(64, activation='relu'),
        Dense(48, activation='relu'),
        Dense(28, activation='relu'),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax", name="dense_output")
    ])
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"  # Conditional loss
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model



def detect_bursts_neurodsp(X, fs=250, bands=[(4, 8), (8, 12), (12, 30), (30, 50)], amp_low=1, amp_high=2, is_binary=None, n_cycles=3):  # Added n_cycles parameter
    """
    Detect bursts using `detect_bursts_dual_threshold` applied per frequency band.

    :param X: Input data of shape (samples, seq_len, n_features)
    :param fs: Sampling rate
    :param bands: Frequency bands for burst detection
    :param amp_low: Lower amplitude threshold
    :param amp_high: Higher amplitude threshold
    :param is_binary: Auto-detect classification type if None
    :param n_cycles: Number of cycles to use for filter design (adjust to shorten filter)
    :return: Binary or multiclass burst predictions (samples, seq_len)
    """
    samples, seq_len, n_features = X.shape

    if is_binary is None:
        is_binary = n_features == 1

    num_bands = len(bands)
    burst_detections = np.zeros((samples, seq_len, num_bands), dtype=int)

    for i, (low_f, high_f) in enumerate(bands):
        f_range = (low_f, high_f)
        for j in range(samples):
            # Pass n_cycles to detect_bursts_dual_threshold to control filter length
            burst_detections[j, :, i] = detect_bursts_dual_threshold(X[j,:,0], fs, (amp_low, amp_high), (low_f, high_f))

    if is_binary:
        return (burst_detections.any(axis=-1)).astype(int)  # Binary: Any band active → 1
    else:
        return np.argmax(burst_detections, axis=-1)  # Multiclass: Assign highest active bandd

import numpy as np

# Step 1: Compute the Hilbert transform amplitude for the signal
def compute_hilbert_features(X, fs=250):  # Accepts 3D input X
    """
    Computes Hilbert amplitude features with frequency-dependent filter orders and smoothing.
    Flattens the input, computes amplitudes, and reshapes back to the original shape.

    Args:
        X (np.ndarray): The input signal data (3D array).
        fs (int, optional): Sampling frequency. Defaults to 250.

    Returns:
        np.ndarray: The Hilbert amplitude features (3D array with the same shape as input X).
    """
    original_shape = X.shape


    # Flatten the input data
    signal_flat = X.flatten()

    # Define frequency bands with filter orders and smoothing factors
    frequency_bands = {
        'theta': (4, 7, 2, 0.0),  # (lowcut, highcut, filter_order, smoothing_factor)
        'alpha': (8, 11, 2, 0.0),
        'beta': (12, 28, 6, 0.3),
        'gamma': (30, 100, 8, 0.4)
    }

    hilbert_features = []

    for band_name, (lowcut, highcut, filter_order, smoothing_factor) in frequency_bands.items():
        # Apply bandpass filter with specified order
        filtered_signal = butter_bandpass_filter(signal_flat, lowcut, highcut, fs, order=filter_order)

        # Compute Hilbert amplitude envelope
        amplitude_envelope = np.abs(hilbert(filtered_signal))

        # Apply smoothing (e.g., moving average) only if smoothing_factor > 0
        if smoothing_factor > 0:
            window_size = int(smoothing_factor * fs)  # Adjust window size based on frequency and smoothing factor
            amplitude_envelope = np.convolve(amplitude_envelope, np.ones(window_size) / window_size, mode='same')
        print(band_name, amplitude_envelope.shape)
        hilbert_features.append(amplitude_envelope)

    # Reshape the features back to the original 3D shape
    hilbert_features_3d = np.column_stack(hilbert_features).reshape(original_shape + (4,))  # Add feature dimension
    print(hilbert_features_3d.shape)
    return hilbert_features_3d

def detect_bursts_hilbert(X, std_factor=1, is_binary=None, fs=250):
    """
    Detect bursts using Hilbert amplitude envelopes with thresholding.

    :param X: Precomputed Hilbert envelopes of shape (samples, seq_len, 4: n_features)
    :param std_factor: Standard deviation factor for thresholding
    :param is_binary: Auto-detect classification type if None
    :return: Binary or multiclass burst predictions (samples, seq_len)
    """
    samples, seq_len, n_features = X.shape

    if n_features == 1:
        X = compute_hilbert_features(X.reshape(samples, seq_len), fs=fs)

    else:
        X = X[..., 1:5] # Filter out the already existing hilbert amplitudes

    # Compute dynamic thresholds for each feature (mean + std_factor * std)
    mean_values = np.mean(X, axis=(0, 1))  # Mean per feature
    std_values = np.std(X, axis=(0, 1))  # Std per feature
    thresholds = mean_values + std_factor * std_values  # Dynamic threshold per feature

    # Detect bursts per feature
    burst_detections = X > thresholds[None, None, :]  # Boolean mask of detected bursts

    # Find the strongest amplitude per timestep across all features
    max_amplitude_band = np.argmax(X, axis=-1)  # (samples, seq_len)

    # Enforce the highest amplitude rule: Remove all bursts except the strongest one
    final_burst_labels = np.zeros((samples, seq_len), dtype=int)  # Default: Noise (0)
    for i in range(samples):
        for t in range(seq_len):
            strongest_band = max_amplitude_band[i, t]  # Identify the strongest amplitude band

            # If this band also crosses the burst threshold, assign it as the final label
            if burst_detections[i, t, strongest_band]:
                final_burst_labels[i, t] = strongest_band + 1  # Shift labels (Theta = 1, Alpha = 2, etc.)

    # ALTERNATIVE (FASTER APPROACH) not tested yet
    # final_burst_labels = np.zeros((samples, seq_len), dtype=int)  # Initialize with 0 (noise)
    # for band_idx in range(X.shape[-1]):  # Iterate over bands (0 to 3 for theta, alpha, beta, gamma)
    #     # Create a mask for where this band is the strongest and crosses the threshold
    #     band_mask = (max_amplitude_band == band_idx) & burst_detections[..., band_idx]

    #     # Assign the band label (band_idx + 1) where the mask is True
    #     final_burst_labels = np.where(band_mask, band_idx + 1, final_burst_labels)


    if is_binary:
        return (final_burst_labels > 0).astype(int)  # Convert to 0/1 for binary classification
    else:
        return final_burst_labels  # Multiclass classification (0-4)

### Multi Models different output layers (Not used)

In [ ]:
# ordinal cross entropy
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

from keras import losses

def ordinal_cross_entropy(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * losses.categorical_crossentropy(y_true, y_pred)


# Define the model
def ordinal_lstm_model(input_shape):
    inputs = Input(shape=input_shape)  # Example input shape (timesteps, features)
    x = Bidirectional(LSTM(32, return_sequences=True))(inputs)
    outputs = Dense(1, activation="linear")(x)  # Single output neuron for ordinal values
    model = Model(inputs, outputs)

    model.compile(optimizer="adam", loss=ordinal_cross_entropy)
    return model

model = ordinal_lstm_model((500,1))

In [ ]:
# Two Output Neurons (Noise vs. Oscillations)

def hybrid_loss(noise_weight=0.5, oscillation_weight=0.5):
    def loss(y_true, y_pred):
        y_true_noise, y_true_oscillation = y_true[..., 0], y_true[..., 1]
        y_pred_noise, y_pred_oscillation = y_pred[..., 0], y_pred[..., 1]

        # Binary crossentropy for noise
        noise_loss = K.binary_crossentropy(y_true_noise, y_pred_noise)

        # Ordinal loss for oscillation
        ordinal_loss = K.abs(y_true_oscillation - y_pred_oscillation) * K.log(K.maximum(K.epsilon(), y_pred_oscillation))


        # Combine losses
        return noise_weight * noise_loss + oscillation_weight * K.mean(ordinal_loss)
    return loss

def hybrid_lstm_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(32, return_sequences=True))(inputs)

    # Two outputs
    noise_output = Dense(1, activation="sigmoid", name="noise")(x)  # Noise vs. oscillations
    oscillation_output = Dense(1, activation="linear", name="oscillation")(x)  # Ordinal output for oscillations

    model = Model(inputs, [noise_output, oscillation_output])

    model.compile(optimizer="adam", loss=hybrid_loss(), metrics=['accuracy'])
    return model

model = hybrid_lstm_model((500,1))

In [ ]:
# Five Output Neurons (Noise Treated Separately)

def multioutput_loss(y_true, y_pred):
    noise_loss = K.binary_crossentropy(y_true[..., 0], y_pred[..., 0])  # Noise
    oscillation_loss = ordinal_cross_entropy(y_true[..., 1:], y_pred[..., 1:])  # Oscillations (1–4)

    # Weighted sum of losses
    return 0.5 * noise_loss + 0.5 * oscillation_loss

def multioutput_lstm_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(32, return_sequences=True))(inputs)

    # Dense layer with 5 outputs
    outputs = Dense(5, activation="softmax")(x)

    model.compile(optimizer="adam", loss=multioutput_loss)
    return Model(inputs, outputs)


model = multioutput_lstm_model((500,1))

In [ ]:
def hierarchical_loss(noise_weight=0.5, oscillation_weight=0.5):
    def loss(y_true, y_pred):
        y_true_noise, y_true_oscillation = y_true[..., 0], y_true[..., 1:]
        y_pred_noise, y_pred_oscillation = y_pred[0], y_pred[1]

        # Binary cross-entropy for Stage 1
        # Reshape y_pred_noise to match y_true_noise
        # Take the mean of predictions over the timesteps before squeeze
        y_pred_noise = tf.reduce_mean(y_pred_noise, axis=1) # Average over timesteps
        y_pred_noise = tf.squeeze(y_pred_noise, axis=-1)  # Squeeze the last dimension of y_pred_noise
        noise_loss = K.binary_crossentropy(y_true_noise, y_pred_noise)

        # Categorical cross-entropy for Stage 2
        # Reshape y_pred_oscillation to match y_true_oscillation if needed
        y_pred_oscillation = tf.reshape(y_pred_oscillation, tf.shape(y_true_oscillation))
        oscillation_loss = K.categorical_crossentropy(y_true_oscillation, y_pred_oscillation)

        # Combine losses
        return noise_weight * noise_loss + oscillation_weight * oscillation_loss
    return loss

# Define the model
def hierarchical_lstm_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(32, return_sequences=True))(inputs)

    # Stage 1: Binary classification (Noise vs. Oscillations)
    noise_output = Dense(1, activation="sigmoid", name="noise")(x)

    # Stage 2: Multi-class classification (Oscillation types)
    oscillation_output = Dense(4, activation="softmax", name="oscillation")(x)

    model = Model(inputs, [noise_output, oscillation_output])


    model.compile(optimizer="adam", loss=hierarchical_loss())
    return model


In [ ]:
# def create_models_ordinal(input_shape):
#     """
#     Creates and compiles all the models, returning them in a dictionary.

#     Args:
#         input_shape: Tuple specifying the input shape for the models.

#     Returns:
#         A dictionary where keys are model names and values are dictionaries
#         containing the model and an empty 'parameters' subkey.
#     """
#     # Use different variable names to avoid conflicts
#     ordinal_lstm = ordinal_lstm_model(input_shape)
#     hybrid_lstm = hybrid_lstm_model(input_shape)
#     multioutput_lstm = multioutput_lstm_model(input_shape)
#     hierarchical_lstm = hierarchical_lstm_model(input_shape)
#     models = {
#         "Ordinal LSTM": {
#             "model": ordinal_lstm,  # Use the renamed variable
#             "parameters": ordinal_lstm.count_params()
#         },
#         "Hierarchical LSTM": {
#             "model": hierarchical_lstm,  # Use the renamed variable
#             "parameters": hierarchical_lstm.count_params()
#         },
#         "Hybrid LSTM": {
#             "model": hybrid_lstm,  # Use the renamed variable
#             "parameters": hybrid_lstm.count_params()
#         },
#         "Multioutput LSTM": {
#             "model": multioutput_lstm,  # Use the renamed variable
#             "parameters": multioutput_lstm.count_params()
#         }
#     }

#     return models

### Create Model Dicts

In [ ]:
def create_models_simple(input_shape, binary, cnn_window_size=100):
    """
    Creates and compiles all the models, returning them in a dictionary.

    Args:
        input_shape: Tuple specifying the input shape for the models.
        binary: Boolean indicating if the problem is binary classification.

    Returns:
        A dictionary where keys are model names and values are dictionaries
        containing the model and an empty 'parameters' subkey.
    """
    lstm_model = create_lstm_baseline(input_shape, binary)
    bilstm_model = create_bidirectional_lstm_balanced(input_shape, binary)
    tcn_model = create_tcn_balanced(input_shape, binary)
    conv_lstm = create_convLSTM_model(input_shape, binary)
    cnn_model = create_cnn_balanced((cnn_window_size, input_shape[1]), binary)
    cnn_timestep_model = create_cnn_per_timestep(input_shape, binary)
    gru_model = create_gru_balanced(input_shape, binary)
    rnn_model = create_rnn_balanced(input_shape, binary)
    fnn_model = create_ffn_balanced(input_shape, binary)
    models = {
        # "Threshold Hilbert": {
        #     "model": detect_bursts_hilbert,
        #     "parameters": 0
        # },
        # "Threshold NeuroDSP": {
        #     "model": detect_bursts_neurodsp,
        #     "parameters": 0
        # },
        # "CNN": {
        #     "model": cnn_model,
        #     "parameters": cnn_model.count_params()
        # },
        # "CNN Timestep": {
        #     "model": cnn_timestep_model,
        #     "parameters": cnn_timestep_model.count_params()
        # },
        "LSTM": {
            "model": lstm_model,
            "parameters": lstm_model.count_params()
        },
        "BiLSTM": {
            "model": bilstm_model,
            "parameters": bilstm_model.count_params()
        },
        "RNN": {
            "model": rnn_model,
            "parameters": rnn_model.count_params()
        },
        "GRU": {
            "model": gru_model,
            "parameters": gru_model.count_params()
        },
        "TCN": {
            "model": tcn_model,
            "parameters": tcn_model.count_params()
        },

        "MLP": {
            "model": fnn_model,
            "parameters": fnn_model.count_params()
        }
    }

    return models

In [ ]:
def model_funcs_simple():
    """
    Creates a dictionary of model creation functions.

    Args:
        input_shape: Tuple specifying the input shape for the models.
        binary: Boolean indicating if the problem is binary classification.

    Returns:
        A dictionary where keys are model names and values are the
        corresponding model creation functions.
    """
    models = {
        # "Threshold NeuroDSP": detect_bursts_neurodsp,
        # "Threshold Hilbert": detect_bursts_hilbert,
        # "CNN": create_cnn_balanced,
        # "CNN Timestep": create_cnn_per_timestep,
        "LSTM": create_lstm_baseline,
        "BiLSTM": create_bidirectional_lstm_balanced,
        "RNN": create_rnn_balanced,
        "GRU": create_gru_balanced,
        "TCN": create_tcn_balanced,  # Uncomment to include TCN
        "MLP": create_ffn_balanced
    }

    return models

In [ ]:
models_siganl_binary = create_models_simple((500, 1), True)
models_singal_multi = create_models_simple((500, 1), False)

models_hilbert_binary = create_models_simple((500, 2), True)
models_hilbert_multi = create_models_simple((500, 5), False)

models_all_features_binary = create_models_simple((500, 7), True)
models_all_features_multi = create_models_simple((500, 25), False)

In [ ]:
import numpy as np  # Make sure NumPy is imported

def combine_and_summarize_models(model_dicts):
  """Combines multiple model dictionaries and summarizes parameters.

  Args:
    model_dicts: A list of dictionaries, where each dictionary represents
      a set of models with their parameters.

  Returns:
    A new dictionary containing all the models and their summarized parameters.
  """
  combined_models = {}
  for model_dict in model_dicts:
    for model_name, model_info in model_dict.items():
      if model_name not in combined_models:
        combined_models[model_name] = {"parameters": []}
      combined_models[model_name]["parameters"].append(model_info["parameters"])

  for model_name, model_info in combined_models.items():
    if model_info["parameters"]:
      combined_models[model_name]["parameters_sum"] = np.mean(
          model_info["parameters"]
      )
      combined_models[model_name]["parameters_std"] = np.std(
          model_info["parameters"]
      )
      del combined_models[model_name]["parameters"]
  return combined_models

all_model_dicts = [models_siganl_binary,
                   models_singal_multi,
                   models_hilbert_binary,
                   models_hilbert_multi,
                   models_all_features_binary,
                   models_all_features_multi]

# Get the summarized parameters
combined_models = combine_and_summarize_models(all_model_dicts)

# Print the results
for model_name, model_info in combined_models.items():
  print(f"Model: {model_name}")
  print(f"  Average Parameters: {model_info['parameters_sum']}")
  print(f"  Standard Deviation of Parameters: {model_info['parameters_std']}\n")

Model: LSTM
  Average Parameters: 5197.666666666667
  Standard Deviation of Parameters: 1106.1510847177353

Model: BiLSTM
  Average Parameters: 5385.666666666667
  Standard Deviation of Parameters: 1520.6093807710408

Model: RNN
  Average Parameters: 5217.833333333333
  Standard Deviation of Parameters: 632.5552457207899

Model: GRU
  Average Parameters: 5201.5
  Standard Deviation of Parameters: 968.1029129178364

Model: TCN
  Average Parameters: 5318.0
  Standard Deviation of Parameters: 519.0028901653632

Model: MLP
  Average Parameters: 5080.333333333333
  Standard Deviation of Parameters: 565.1261412778173



In [ ]:
print(models_siganl_binary)
print(models_singal_multi)
print(models_hilbert_binary)
print(models_hilbert_multi)
print(models_all_features_binary)
print(models_all_features_multi)

{'LSTM': {'model': <Sequential name=sequential_186, built=True>, 'parameters': 4385}, 'BiLSTM': {'model': <Sequential name=sequential_187, built=True>, 'parameters': 4269}, 'RNN': {'model': <Sequential name=sequential_193, built=True>, 'parameters': 4691}, 'GRU': {'model': <Sequential name=sequential_192, built=True>, 'parameters': 4478}, 'TCN': {'model': <Sequential name=sequential_188, built=True>, 'parameters': 4936}, 'MLP': {'model': <Sequential name=sequential_194, built=True>, 'parameters': 4649}}
{'LSTM': {'model': <Sequential name=sequential_195, built=True>, 'parameters': 4517}, 'BiLSTM': {'model': <Sequential name=sequential_196, built=True>, 'parameters': 4449}, 'RNN': {'model': <Sequential name=sequential_202, built=True>, 'parameters': 4963}, 'GRU': {'model': <Sequential name=sequential_201, built=True>, 'parameters': 4630}, 'TCN': {'model': <Sequential name=sequential_197, built=True>, 'parameters': 5000}, 'MLP': {'model': <Sequential name=sequential_203, built=True>, 'pa

## Data Import

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Including Features/5_class_mid_noise_30s_features_vec.npy"
label_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Including Features/5_class_mid_noise_30s_numeric_label_vec.npy"

data_vec, label_vec = du.load_data(data_path, label_path)

du.data_info(data_vec)

DATASET INFO:
Shape: (10, 20, 5, 7500, 25)

No of Samples: 10

No of Frequencies: 20
Freqency values: [np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(11), np.int64(13), np.int64(16), np.int64(18), np.int64(22), np.int64(26), np.int64(31), np.int64(36), np.int64(43), np.int64(51), np.int64(60), np.int64(71), np.int64(84), np.int64(100)]

No of noise ratios: 5
Signal to Noise ratios (in db) [np.int64(-10), np.int64(-8), np.int64(-6), np.int64(-4), np.int64(-2), np.int64(0), np.int64(2), np.int64(4), np.int64(6), np.int64(8)]

No of Datapoints: 7500

No of Features per Datapoint: 25 (signal, hilbert amp, 20 wavelets for each freq)


In [ ]:
data_vec_trim = data_vec[:, :, :]
label_vec_trim = label_vec[:, :, :]

print(data_vec_trim.shape)
print(label_vec_trim.shape)

(10, 20, 5, 7500, 25)
(10, 20, 5, 7500)


In [ ]:
import numpy as np

def split_sequences_keep_data(X, y, new_seq_len):
    """
    Splits input sequences and their per-timestep labels into smaller sequences of length `new_seq_len`.

    Args:
        X (numpy array): Input data of shape (samples, seq_len, features).
        y (numpy array): Labels of shape (samples, seq_len) or (samples, seq_len, ...).
        new_seq_len (int): Desired sequence length.

    Returns:
        X_new: New input data with shape (new_samples, new_seq_len, features).
        y_new: New labels with shape (new_samples, new_seq_len, ...).
    """
    num_samples, original_seq_len, num_features = X.shape
    if original_seq_len % new_seq_len != 0:
        print(f"Warning: Original seq_len ({original_seq_len}) is not a multiple of new_seq_len ({new_seq_len}).")

    # Number of new samples per original sequence
    num_segments = original_seq_len // new_seq_len

    # Create new datasets by reshaping
    X_new = np.reshape(X[:, :num_segments * new_seq_len, :],
                       (-1, new_seq_len, num_features))  # Shape: (samples * num_segments, new_seq_len, features)
    y_new = np.reshape(y[:, :num_segments * new_seq_len, ...],
                       (-1, new_seq_len, *y.shape[2:]))  # Adjust labels to match input shape

    return X_new, y_new


In [ ]:
data_vec_shaped, label_vec_shaped = du.preprocess_data(data_vector=data_vec_trim, label_vector=label_vec_trim)

print(data_vec_shaped.shape)
print(label_vec_shaped.shape)

(1000, 7500, 25)
(1000, 7500, 1)


In [ ]:
new_seq_len = 500 # 7500 is origianl length

sample_increase_factor = int(data_vec_shaped.shape[1] / new_seq_len)

data_vec_reshaped, label_vec_reshaped = split_sequences_keep_data(data_vec_shaped, label_vec_shaped, new_seq_len=new_seq_len)

# Extract noise levels
noise_levels = np.arange(data_vec_trim.shape[2])  # Assuming noise levels are 0, 1, 2, 3, 4
noise_levels = np.repeat(noise_levels, data_vec_trim.shape[0] * data_vec_trim.shape[1] * sample_increase_factor)

In [ ]:

print(data_vec_reshaped.shape)
print(label_vec_reshaped.shape)
print(noise_levels.shape)

(15000, 500, 25)
(15000, 500, 1)
(15000,)


## Data Splits

Be careful about choosing binary or not in the TT Splits

In [ ]:
# 1. Multi Models:
wavelet_remove_index = [1,2,3,4]
all_indices = np.arange(data_vec_reshaped.shape[-1])
indices_to_keep = np.delete(all_indices, wavelet_remove_index)

hilbert_data_vec_multi = data_vec_reshaped[:,:,:5]
signal_data_vec_multi = data_vec_reshaped[:,:, :1]
wavelet_data_vec_multi = data_vec_reshaped[:,:, indices_to_keep]
all_data_vec_multi = data_vec_reshaped

label_multi = label_vec_reshaped

print(hilbert_data_vec_multi.shape)
print(signal_data_vec_multi.shape)
print(wavelet_data_vec_multi.shape)
print(all_data_vec_multi.shape)
print(label_multi.shape)


(15000, 500, 5)
(15000, 500, 1)
(15000, 500, 21)
(15000, 500, 25)
(15000, 500, 1)


In [ ]:
# 2. Binary Models:

data_vec_beta_trim = data_vec[:, 10:15, :]
label_vec_beta_trim = label_vec[:, 10:15, :]

data_vec_shaped, label_vec_shaped = du.preprocess_data(data_vector=data_vec_beta_trim, label_vector=label_vec_beta_trim)

print(data_vec_shaped.shape)
print(label_vec_shaped.shape)

new_seq_len = 500 # 7500 is origianl length

sample_increase_factor = int(data_vec_shaped.shape[1] / new_seq_len)

data_vec_reshaped, label_vec_reshaped = split_sequences_keep_data(data_vec_shaped, label_vec_shaped, new_seq_len=new_seq_len)

print(data_vec_reshaped.shape)
print(label_vec_reshaped.shape)

beta_hilbert_incl_index = [0,3]
beta_hilbert_only_index = [3]
beta_wavelet_incl_index = [0,12,13,14,15,16]
beta_wavelet_only_index = [12,13,14,15,16]
beta_allF_index         = beta_hilbert_incl_index + beta_wavelet_only_index

hilbert_data_vec_binary = data_vec_reshaped[:,:,beta_hilbert_incl_index]
signal_data_vec_binary = data_vec_reshaped[:,:, :1]
wavelet_data_vec_binary = data_vec_reshaped[:,:, beta_wavelet_incl_index]
all_data_vec_binary = data_vec_reshaped[:,:,beta_allF_index]

label_binary = label_vec_reshaped.copy()
label_binary[label_binary >= 1] = 1  # Convert to binary labels

print(hilbert_data_vec_binary.shape)
print(signal_data_vec_binary.shape)
print(wavelet_data_vec_binary.shape)
print(all_data_vec_binary.shape)
print(label_binary.shape)


(250, 7500, 25)
(250, 7500, 1)
(3750, 500, 25)
(3750, 500, 1)
(3750, 500, 2)
(3750, 500, 1)
(3750, 500, 6)
(3750, 500, 7)
(3750, 500, 1)


## Train Models

In [ ]:
def evaluate_model(model, X_train, y_train, X_val, y_val, binary):
    """
    Trains and evaluates the model, returning the metrics and evaluation data.
    """


    model.summary()

    start_time = time.time()

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1, validation_data=(X_val, y_val))

    # Measure training time
    training_time = time.time() - start_time

    start_time = time.time()

    # Evaluate
    y_pred = model.predict(X_val).argmax(axis=-1) if not binary else (model.predict(X_val) > 0.5).astype(int)
    inference_time = time.time() - start_time

    y_pred = y_pred.flatten()
    y_val = y_val.flatten()


    accuracy = accuracy_score(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    recall = recall_score(y_val, y_pred, average="binary" if binary else "macro")
    f1 = f1_score(y_val, y_pred, average="binary" if binary else "macro")
    cm = confusion_matrix(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"MAE: {mae}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"MCC: {mcc}")

    return model , {
        "accuracy": accuracy,
        "mcc": mcc,
        "mae": mae,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": cm,
        "training_time": training_time,
        "inference_time": inference_time

    }


In [ ]:
def train_and_evaluate_models(models, X_train, y_train, X_val, y_val, binary):
    """
    Trains and evaluates all models in the dictionary, storing performance metrics.

    Args:
        models: Dictionary containing models with subkeys "model" and "parameters".
        X_train: Training data.
        y_train: Training labels.
        X_val: Validation data.
        y_val: Validation labels.

    Returns:
        The updated models dictionary with a new "performances" subkey for each model.
    """

    for model_name, model_data in models.items():#
        print(f"Training and evaluating {model_name}...")  # Optional: Print progress
        model = model_data["model"]  # Get the model object

        trained_model, performance_metrics = evaluate_model(model, X_train, y_train, X_val, y_val, binary)

        # Save the trained model in Dict
        model_data["model"] = trained_model

        # Store the performance metrics in the "performances" subkey
        model_data["performances"] = performance_metrics

    return models  # Return the updated models dictionary

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

NameError: name 'X_train' is not defined

In [ ]:
# Using Only the signal as an input feature

signal_only_models = train_and_evaluate_models(models, X_train, y_train, X_test, y_test, binary=False)

In [ ]:
models


{'LSTM': {'model': <keras.engine.sequential.Sequential at 0x78eb28a7b850>,
  'parameters': 4385,
  'performances': {'accuracy': 0.8542,
   'mcc': 0.70743184074071,
   'mae': 0.1458,
   'recall': 0.8183711048158641,
   'f1': 0.8408574921411107,
   'confusion_matrix': array([[70353,  9047],
          [12823, 57777]]),
   'training_time': 144.01327061653137,
   'inference_time': 0.8876395225524902}},
 'BiLSTM': {'model': <keras.engine.sequential.Sequential at 0x78eb2929d150>,
  'parameters': 5041,
  'performances': {'accuracy': 0.8995066666666667,
   'mcc': 0.7982790522855711,
   'mae': 0.10049333333333334,
   'recall': 0.8911756373937677,
   'f1': 0.8930223975927556,
   'confusion_matrix': array([[72009,  7391],
          [ 7683, 62917]]),
   'training_time': 153.8624861240387,
   'inference_time': 1.3268213272094727}},
 'RNN': {'model': <keras.engine.sequential.Sequential at 0x78eb2b4e1090>,
  'parameters': 4289,
  'performances': {'accuracy': 0.83588,
   'mcc': 0.6710292799915758,
   '

In [ ]:
from IPython.display import HTML

def display_models_with_performances(models):
    """
    Displays models with existing 'performances' in an HTML table.

    Args:
        models: A dictionary where keys are model names and values are dictionaries
               containing model information, including 'performances' if available.
    """

    html = """
    <style>
    table {
        border-collapse: collapse;
        width: 100%;
    }
    th, td {
        text-align: center;
        padding: 8px;
        border: 1px solid #ddd;
    }
    th {
        background-color: #4f50b3;
    }
    </style>
    <table><thead><tr><th>Model</th><th>Parameters</th><th>Accuracy</th><th>MCC</th><th>Recall</th><th>F1</th><th>Training Time</th><th>Inference Time</th></tr></thead><tbody>"""

    for model_name, model_data in models.items():
        if "performances" in model_data:
            performances = model_data["performances"]
            html += f"<tr><td>{model_name}</td>"
            html += f"<td>{model_data['parameters']}</td>"
            html += f"<td>{performances['accuracy']:.3f}</td>"
            html += f"<td>{performances['mcc']:.3f}</td>"
            html += f"<td>{performances['recall']:.3f}</td>"
            html += f"<td>{performances['f1']:.3f}</td>"
            html += f"<td>{performances['training_time']:.1f}</td>"
            html += f"<td>{performances['inference_time']:.3f}</td></tr>"

    html += "</tbody></table>"
    display(HTML(html))

display_models_with_performances(models)

## K-Fold Analysis

- Stratified k-fold
- Make function that takes in only the model_dict and returns result dict
- Create Heatlike result visualizations
-

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import matthews_corrcoef

class MCCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, is_binary=True):
        super().__init__()
        self.validation_data = validation_data  # Tuple (x_val, y_val)
        self.is_binary = is_binary

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        x_val, y_val = self.validation_data
        y_pred = self.model.predict(x_val, verbose=0)

        # Ensure y_val is flattened (shape: (N,))
        y_true = y_val.flatten()

        if self.is_binary:
            # Binary classification: round predictions
            y_pred = np.round(y_pred).flatten()
        else:
            # Multiclass classification: use argmax (since predictions are (N, 5) softmax outputs)
            y_pred = np.argmax(y_pred, axis=-1).flatten()

        # Compute MCC
        mcc = matthews_corrcoef(y_true, y_pred)

        # Log MCC
        logs["mcc"] = mcc
        print(f"\nEpoch {epoch + 1} - MCC: {mcc:.4f}")



In [ ]:

def stratified_kfold_split(X, y, num_folds=5):
    """
    Performs Stratified K-Fold cross-validation on sequence data.

    Args:
        X: ndarray of shape (samples, timesteps, features) -> (7500, 500, ?)
        y: ndarray of shape (samples, timesteps, 1) -> (7500, 500, 1)
        num_folds: Number of folds for cross-validation.

    Returns:
        List of (train_idx, val_idx) tuples.
    """
    # Convert y from (7500, 500, 1) → (7500, 500)
    y_flat = y[:, :, 0]  # Remove last dimension

    # Get **max class per sequence**
    y_max = np.max(y_flat, axis=1)  # Shape → (7500,)

    # Stratified K-Fold splitting
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    folds = list(skf.split(X, y_max))  # Get (train_idx, val_idx) splits

    return folds


In [ ]:
from collections import defaultdict

# Train and Evaluate Function
def train_and_evaluate(models, X, y, k_folds=5, batch_size=16, epochs=10, basePath="/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/", dir_name="hilbert_binary"):

    skf_folds = stratified_kfold_split(X,y,num_folds=k_folds)

    # Determines whether we work with binary data or multi data
    binary = len(np.unique(y)) == 2  # Check if only 2 unique values in y
    print(f"Binary: {binary}")


    model_results = {model_name: {} for model_name in models.keys()}
    trained_models = {model_name: [] for model_name in models.keys()}

    for model_name, model_fn in models.items():
        acc_scores, mcc_scores, train_times, infer_times = [], [], [], []

        # *** Initialize a list to store history for each fold ***
        val_mcc = []
        val_accuracy = []
        val_losses = []
        training_losses = []
        training_accuracy = []
        num_params = None


        for fold, (train_idx, val_idx) in enumerate(skf_folds):

            print(f"Training Model: {model_name}, Fold: {fold+1}")

            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]



            val_mccs = None # Initialize to None for Threshold case

            mcc_callback = MCCCallback(validation_data=(X_val, y_val), is_binary=binary)

            if model_name == "CNN":
                model = model_fn(input_shape=(100, X_train.shape[-1]), binary=binary)

                 # Create a small subset of data for estimation (e.g., 10%)
                subset_size = int(0.5 * len(X))

                train_gen_100 = SlidingWindowGenerator(X_train[:subset_size], y_train[:subset_size], window_size=100, batch_size=batch_size*32, shuffle=False)
                val_gen_100 = SlidingWindowGenerator(X_val[:subset_size], y_val[:subset_size], window_size=100, batch_size=batch_size*32, shuffle=False)

                start_train = time.time()
                history = model.fit(train_gen_100, epochs=epochs, batch_size=batch_size*32, validation_data=val_gen_100,
                                    verbose=1)
                train_time = time.time() - start_train

                start_time = time.time()
                y_true = []
                y_pred = []
                for i in range(len(val_gen_100)):
                    X_batch, y_batch = val_gen_100[i]  # Get a batch of data
                    y_true.extend(y_batch)
                    y_pred_batch = model.predict(X_batch, verbose=0).argmax(axis=-1) if not binary else (model.predict(X_batch) > 0.5).astype(int)
                    y_pred.extend(y_pred_batch)
                infer_time = time.time() - start_time

                 # Convert y_true_flat and y_pred_flat to NumPy arrays
                y_true_flat = np.array(y_true).flatten()
                y_pred_flat = np.array(y_pred).flatten()

                training_accs = history.history['accuracy']
                val_accs = history.history['val_accuracy']
                val_losses = history.history['val_loss']
                training_loss = history.history['loss']

                print(np.unique(y_true_flat))
                print(np.unique(y_pred_flat))
                print(len(y_true_flat))
                print(len(y_pred_flat))
                print(y_true_flat.shape)
                print(y_pred_flat.shape)

            elif model_name in ["Threshold NeuroDSP", "Threshold Hilbert"]:
                # Apply neurodsp burst detection
                start_infer = time.time()
                y_true = y_val
                y_pred = model_fn(X_val, is_binary=binary)
                train_time = 0.0
                num_params = "N/A"
                infer_time = (time.time() - start_infer) / len(X_val)
                y_true_flat = y_true.flatten()
                y_pred_flat = y_pred.flatten()



            else:
                model = model_fn(input_shape=X_train.shape[1:], binary=binary)
                start_train = time.time()
                history = model.fit(X_train, y_train, epochs=epochs,
                                    batch_size=batch_size, validation_data=(X_val, y_val),
                                    verbose=1, callbacks=[mcc_callback])
                train_time = time.time() - start_train

                start_infer = time.time()
                y_true = y_val
                y_pred = model.predict(X_val).argmax(axis=-1) if not binary else (model.predict(X_val) > 0.5).astype(int)
                infer_time = (time.time() - start_infer) / len(X_val)

                val_mccs = history.history['mcc']  # Get the validation MCC values

                y_true_flat = y_true.flatten()
                y_pred_flat = y_pred.flatten()
                training_accs = history.history['accuracy']
                val_accs = history.history['val_accuracy']
                val_losses = history.history['val_loss']
                training_loss = history.history['loss']

            acc_scores.append(accuracy_score(y_true_flat, y_pred_flat))
            mcc_scores.append(matthews_corrcoef(y_true_flat, y_pred_flat))

            infer_times.append(infer_time)
            train_times.append(train_time)

            if model_name not in ["Threshold NeuroDSP", "Threshold Hilbert"]:

                if val_mccs:
                  val_mcc.append(val_mccs)
                val_accuracy.append(val_accs)
                training_accuracy.append(training_accs)
                training_losses.append(training_loss)

                num_params = model.count_params()

                model_save_path = f"{basePath}/{dir_name}/{model_name}/{model_name}_fold_{fold+1}.h5"
                model.save(model_save_path)
                trained_models[model_name].append(model_save_path)



        model_results[model_name] = {
            "Accuracy": {"mean": np.mean(acc_scores), "std": np.std(acc_scores), "min": np.min(acc_scores), "max": np.max(acc_scores)},
            "MCC": {"mean": np.mean(mcc_scores), "std": np.std(mcc_scores), "min": np.min(mcc_scores), "max": np.max(mcc_scores)},
            "Train Time (s)": {"mean": np.mean(train_times), "std": np.std(train_times), "min": np.min(train_times), "max": np.max(train_times)} if train_times else None,
            "Inference Time (s/sample)": {"mean": np.mean(infer_times), "std": np.std(infer_times), "min": np.min(infer_times), "max": np.max(infer_times)},
            "Training Accuracy": training_accuracy,
            "Training Loss": training_losses,
            "Validation MCC": val_mcc,
            "Validation Accuracy": val_accuracy,
            "Validation Loss": val_losses,
            "Parameters": num_params
        }

        pprint(model_results[model_name], width=1)

        if model_name not in ["Threshold NeuroDSP", "Threshold Hilbert"]:
            final_model_save_path = f"{basePath}/{dir_name}/{model_name}/{model_name}_final.h5"
            model.save(final_model_save_path)

    return model_results, trained_models



In [ ]:
simple_models_dict = model_funcs_simple()

In [ ]:
print(simple_models_dict.keys())

dict_keys(['LSTM', 'BiLSTM', 'RNN', 'GRU', 'TCN', 'MLP'])


In [ ]:
basePath= "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/Cross-Model Comparison/"

In [ ]:
signal_data_vec_binary.shape

(3750, 500, 1)

### Binary Models


In [ ]:
# Signal Binary

model_results, trained_models = train_and_evaluate(simple_models_dict, X=signal_data_vec_binary, y=label_binary,
                                                   epochs=20, batch_size=32,
                                                   basePath=basePath, dir_name='signal_binary')

filePath = f"{basePath}/Signal_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

Binary: True
Training Model: LSTM, Fold: 1
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6222 - loss: 0.6540
Epoch 1 - MCC: 0.6589
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.6230 - loss: 0.6532 - val_accuracy: 0.8299 - val_loss: 0.3922 - mcc: 0.6589
Epoch 2/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8403 - loss: 0.3722
Epoch 2 - MCC: 0.7232
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.8405 - loss: 0.3717 - val_accuracy: 0.8620 - val_loss: 0.3272 - mcc: 0.7232
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8659 - loss: 0.3194
Epoch 3 - MCC: 0.7517
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8659 - loss: 0.3193 - val_accuracy: 0.8760 - val_loss: 0.2986 - mcc: 0.7517
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8792 - loss: 0.2906
Epoch 4 - MCC: 0.7648
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.8792 - loss: 0.2906 - val_accuracy: 0.8827 - val_loss: 0.2829 - mcc: 0.7648
Epoch 5/2

Training Model: LSTM, Fold: 2
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6246 - loss: 0.6448
Epoch 1 - MCC: 0.6283
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.6254 - loss: 0.6440 - val_accuracy: 0.8142 - val_loss: 0.4345 - mcc: 0.6283
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8213 - loss: 0.4084
Epoch 2 - MCC: 0.6917
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8214 - loss: 0.4082 - val_accuracy: 0.8464 - val_loss: 0.3566 - mcc: 0.6917
Epoch 3/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8491 - loss: 0.3490
Epoch 3 - MCC: 0.7296
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.8492 - loss: 0.3487 - val_accuracy: 0.8650 - val_loss: 0.3193 - mcc: 0.7296
Epoch 4/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8664 - loss: 0.3163
Epoch 4 - MCC: 0.7566
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8665 - loss: 0.3161 - val_accuracy: 0.8787 - val_loss: 0.2920 - mcc: 0.7566
Epoch 5/20
91/94 ━━━━━

Training Model: LSTM, Fold: 3
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6156 - loss: 0.6548
Epoch 1 - MCC: 0.6227
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.6164 - loss: 0.6540 - val_accuracy: 0.8121 - val_loss: 0.4403 - mcc: 0.6227
Epoch 2/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8242 - loss: 0.4069
Epoch 2 - MCC: 0.6912
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8245 - loss: 0.4062 - val_accuracy: 0.8462 - val_loss: 0.3558 - mcc: 0.6912
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8573 - loss: 0.3365
Epoch 3 - MCC: 0.7418
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.8574 - loss: 0.3364 - val_accuracy: 0.8714 - val_loss: 0.3104 - mcc: 0.7418
Epoch 4/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8791 - loss: 0.2946
Epoch 4 - MCC: 0.7542
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8791 - loss: 0.2946 - val_accuracy: 0.8775 - val_loss: 0.2941 - mcc: 0.7542
Epoch 5/20
92/94 ━━━━━

Training Model: LSTM, Fold: 4
Epoch 1/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6364 - loss: 0.6426
Epoch 1 - MCC: 0.6355
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.6380 - loss: 0.6410 - val_accuracy: 0.8184 - val_loss: 0.4178 - mcc: 0.6355
Epoch 2/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8259 - loss: 0.3992
Epoch 2 - MCC: 0.7051
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8262 - loss: 0.3986 - val_accuracy: 0.8530 - val_loss: 0.3542 - mcc: 0.7051
Epoch 3/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8562 - loss: 0.3395
Epoch 3 - MCC: 0.7282
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8562 - loss: 0.3394 - val_accuracy: 0.8642 - val_loss: 0.3232 - mcc: 0.7282
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8684 - loss: 0.3137
Epoch 4 - MCC: 0.7508
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8684 - loss: 0.3137 - val_accuracy: 0.8757 - val_loss: 0.2975 - mcc: 0.7508
Epoch 5/20
94/94 ━━━━━

Training Model: LSTM, Fold: 5
Epoch 1/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6307 - loss: 0.6469
Epoch 1 - MCC: 0.6585
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.6333 - loss: 0.6443 - val_accuracy: 0.8300 - val_loss: 0.3993 - mcc: 0.6585
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8375 - loss: 0.3775
Epoch 2 - MCC: 0.7115
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8376 - loss: 0.3774 - val_accuracy: 0.8563 - val_loss: 0.3369 - mcc: 0.7115
Epoch 3/20
91/94 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8645 - loss: 0.3221
Epoch 3 - MCC: 0.7448
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.8646 - loss: 0.3218 - val_accuracy: 0.8729 - val_loss: 0.3027 - mcc: 0.7448
Epoch 4/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8772 - loss: 0.2948
Epoch 4 - MCC: 0.7569
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8772 - loss: 0.2948 - val_accuracy: 0.8787 - val_loss: 0.2877 - mcc: 0.7569
Epoch 5/20
93/94 ━━━━━

...

Training Model: MLP, Fold: 2
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6243 - loss: 0.6575
Epoch 1 - MCC: 0.3442
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.6246 - loss: 0.6573 - val_accuracy: 0.6732 - val_loss: 0.6103 - mcc: 0.3442
Epoch 2/20
90/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6702 - loss: 0.6119
Epoch 2 - MCC: 0.3474
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6702 - loss: 0.6120 - val_accuracy: 0.6731 - val_loss: 0.6105 - mcc: 0.3474
Epoch 3/20
92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6744 - loss: 0.6090
Epoch 3 - MCC: 0.3459
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6743 - loss: 0.6091 - val_accuracy: 0.6730 - val_loss: 0.6104 - mcc: 0.3459
Epoch 4/20
89/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6731 - loss: 0.6103
Epoch 4 - MCC: 0.3470
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6729 - loss: 0.6104 - val_accuracy: 0.6731 - val_loss: 0.6105 - mcc: 0.3470
Epoch 5/20
93/94 ━━━━━━━━━━━━

Training Model: MLP, Fold: 3
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6237 - loss: 0.6515
Epoch 1 - MCC: 0.3314
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6241 - loss: 0.6512 - val_accuracy: 0.6669 - val_loss: 0.6151 - mcc: 0.3314
Epoch 2/20
91/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6679 - loss: 0.6141
Epoch 2 - MCC: 0.3318
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6680 - loss: 0.6140 - val_accuracy: 0.6668 - val_loss: 0.6155 - mcc: 0.3318
Epoch 3/20
77/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6721 - loss: 0.6103
Epoch 3 - MCC: 0.3276
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6720 - loss: 0.6105 - val_accuracy: 0.6659 - val_loss: 0.6157 - mcc: 0.3276
Epoch 4/20
78/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6714 - loss: 0.6109
Epoch 4 - MCC: 0.3329
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6715 - loss: 0.6109 - val_accuracy: 0.6665 - val_loss: 0.6157 - mcc: 0.3329
Epoch 5/20
88/94 ━━━━━━━━━━━━

Training Model: MLP, Fold: 4
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6049 - loss: 0.6524
Epoch 1 - MCC: 0.3391
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.6053 - loss: 0.6522 - val_accuracy: 0.6703 - val_loss: 0.6123 - mcc: 0.3391
Epoch 2/20
77/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6739 - loss: 0.6094
Epoch 2 - MCC: 0.3399
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6734 - loss: 0.6099 - val_accuracy: 0.6707 - val_loss: 0.6114 - mcc: 0.3399
Epoch 3/20
93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6730 - loss: 0.6093
Epoch 3 - MCC: 0.3432
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6730 - loss: 0.6093 - val_accuracy: 0.6687 - val_loss: 0.6140 - mcc: 0.3432
Epoch 4/20
89/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6747 - loss: 0.6096
Epoch 4 - MCC: 0.3395
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6745 - loss: 0.6098 - val_accuracy: 0.6706 - val_loss: 0.6116 - mcc: 0.3395
Epoch 5/20
85/94 ━━━━━━━━━━━━

Training Model: MLP, Fold: 5
Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5997 - loss: 0.6546
Epoch 1 - MCC: 0.3429
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.6002 - loss: 0.6544 - val_accuracy: 0.6703 - val_loss: 0.6118 - mcc: 0.3429
Epoch 2/20
91/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6709 - loss: 0.6122
Epoch 2 - MCC: 0.3382
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6709 - loss: 0.6122 - val_accuracy: 0.6709 - val_loss: 0.6113 - mcc: 0.3382
Epoch 3/20
77/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6708 - loss: 0.6120
Epoch 3 - MCC: 0.3407
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6710 - loss: 0.6120 - val_accuracy: 0.6711 - val_loss: 0.6108 - mcc: 0.3407
Epoch 4/20
89/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6693 - loss: 0.6135
Epoch 4 - MCC: 0.3395
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6694 - loss: 0.6134 - val_accuracy: 0.6713 - val_loss: 0.6109 - mcc: 0.3395
Epoch 5/20
90/94 ━━━━━━━━━━━━

{'Accuracy': {'max': np.float64(0.673032),
              'mean': np.float64(0.6708309333333333),
              'min': np.float64(0.6664666666666667),
              'std': np.float64(0.002396516345772658)},
 'Inference Time (s/sample)': {'max': np.float64(0.0003018061319986979),
                               'mean': np.float64(0.0002565639495849609),
                               'min': np.float64(0.0001734301249186198),
                               'std': np.float64(4.4827774278185976e-05)},
 'MCC': {'max': np.float64(0.34508052552965757),
         'mean': np.float64(0.3397109407932288),
         'min': np.float64(0.32995444704069504),
         'std': np.float64(0.005528070869962765)},
 'Parameters': 4649,
 'Train Time (s)': {'max': np.float64(24.03471541404724),
                    'mean': np.float64(19.505299663543703),
                    'min': np.float64(17.112409591674805),
                    'std': np.float64(2.403096235557626)},
 'Training Accuracy': [[0.645784854888916,
 

In [ ]:
# Hilbert Binary

model_results, trained_models = train_and_evaluate(simple_models_dict, X=hilbert_data_vec_binary, y=label_binary, epochs=20, basePath=basePath, dir_name='hilbert_binary')

filePath = f"{basePath}/Hilbert_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

Binary: True
Training Model: LSTM, Fold: 1
Epoch 1/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7250 - loss: 0.5030
Epoch 1 - MCC: 0.7848
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.7261 - loss: 0.5017 - val_accuracy: 0.8924 - val_loss: 0.2573 - mcc: 0.7848
Epoch 2/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9007 - loss: 0.2378
Epoch 2 - MCC: 0.8128
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9007 - loss: 0.2377 - val_accuracy: 0.9064 - val_loss: 0.2251 - mcc: 0.8128
Epoch 3/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9062 - loss: 0.2196
Epoch 3 - MCC: 0.8135
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9062 - loss: 0.2196 - val_accuracy: 0.9068 - val_loss: 0.2168 - mcc: 0.8135
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9101 - loss: 0.2104
Epoch 4 - MCC: 0.8256
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9101 - loss: 0.2104 - val_accuracy: 0.9130 - val_loss: 0.2054 - mcc: 

Training Model: LSTM, Fold: 2
Epoch 1/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7238 - loss: 0.5249
Epoch 1 - MCC: 0.7779
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7248 - loss: 0.5234 - val_accuracy: 0.8891 - val_loss: 0.2652 - mcc: 0.7779
Epoch 2/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8928 - loss: 0.2544
Epoch 2 - MCC: 0.7972
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8928 - loss: 0.2542 - val_accuracy: 0.8988 - val_loss: 0.2352 - mcc: 0.7972
Epoch 3/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9032 - loss: 0.2268
Epoch 3 - MCC: 0.8063
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9032 - loss: 0.2268 - val_accuracy: 0.9033 - val_loss: 0.2227 - mcc: 0.8063
Epoch 4/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9059 - loss: 0.2167
Epoch 4 - MCC: 0.8222
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9059 - loss: 0.2167 - val_accuracy: 0.9114 - val_loss: 0.2066 - mcc: 0.8222
Epoch 

Training Model: LSTM, Fold: 3
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7458 - loss: 0.4925
Epoch 1 - MCC: 0.7905
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.7462 - loss: 0.4919 - val_accuracy: 0.8956 - val_loss: 0.2543 - mcc: 0.7905
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9017 - loss: 0.2351
Epoch 2 - MCC: 0.8013
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9016 - loss: 0.2351 - val_accuracy: 0.9008 - val_loss: 0.2320 - mcc: 0.8013
Epoch 3/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9032 - loss: 0.2279
Epoch 3 - MCC: 0.8166
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9032 - loss: 0.2278 - val_accuracy: 0.9086 - val_loss: 0.2160 - mcc: 0.8166
Epoch 4/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9088 - loss: 0.2148
Epoch 4 - MCC: 0.8160
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9088 - loss: 0.2148 - val_accuracy: 0.9082 - val_loss: 0.2149 - mcc: 0.8160
Epoch 

Training Model: LSTM, Fold: 4
Epoch 1/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7189 - loss: 0.5063
Epoch 1 - MCC: 0.7936
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7210 - loss: 0.5035 - val_accuracy: 0.8971 - val_loss: 0.2454 - mcc: 0.7936
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8939 - loss: 0.2487
Epoch 2 - MCC: 0.8034
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8939 - loss: 0.2487 - val_accuracy: 0.9019 - val_loss: 0.2338 - mcc: 0.8034
Epoch 3/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9009 - loss: 0.2308
Epoch 3 - MCC: 0.8093
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9009 - loss: 0.2308 - val_accuracy: 0.9048 - val_loss: 0.2242 - mcc: 0.8093
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9036 - loss: 0.2238
Epoch 4 - MCC: 0.8130
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9036 - loss: 0.2238 - val_accuracy: 0.9062 - val_loss: 0.2204 - mcc: 0.8130
Epoch 

Training Model: LSTM, Fold: 5
Epoch 1/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7372 - loss: 0.5277
Epoch 1 - MCC: 0.7842
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.7380 - loss: 0.5263 - val_accuracy: 0.8925 - val_loss: 0.2567 - mcc: 0.7842
Epoch 2/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8963 - loss: 0.2471
Epoch 2 - MCC: 0.8079
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8964 - loss: 0.2471 - val_accuracy: 0.9041 - val_loss: 0.2261 - mcc: 0.8079
Epoch 3/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9008 - loss: 0.2323
Epoch 3 - MCC: 0.8105
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9009 - loss: 0.2322 - val_accuracy: 0.9056 - val_loss: 0.2176 - mcc: 0.8105
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9080 - loss: 0.2163
Epoch 4 - MCC: 0.8211
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9080 - loss: 0.2163 - val_accuracy: 0.9108 - val_loss: 0.2102 - mcc: 0.8211
Epoch 

...

Training Model: MLP, Fold: 2
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6931 - loss: 0.5198
Epoch 1 - MCC: 0.6898
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6936 - loss: 0.5192 - val_accuracy: 0.8446 - val_loss: 0.3292 - mcc: 0.6898
Epoch 2/20
183/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8458 - loss: 0.3272
Epoch 2 - MCC: 0.6930
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8459 - loss: 0.3270 - val_accuracy: 0.8467 - val_loss: 0.3223 - mcc: 0.6930
Epoch 3/20
179/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8494 - loss: 0.3192
Epoch 3 - MCC: 0.6993
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8493 - loss: 0.3194 - val_accuracy: 0.8501 - val_loss: 0.3169 - mcc: 0.6993
Epoch 4/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8488 - loss: 0.3194
Epoch 4 - MCC: 0.6971
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8488 - loss: 0.3194 - val_accuracy: 0.8490 - val_loss: 0.3174 - mcc: 0.6971
Epoch 5/20
182

Training Model: MLP, Fold: 3
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6829 - loss: 0.5484
Epoch 1 - MCC: 0.6871
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6834 - loss: 0.5478 - val_accuracy: 0.8439 - val_loss: 0.3298 - mcc: 0.6871
Epoch 2/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8517 - loss: 0.3169
Epoch 2 - MCC: 0.6862
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8517 - loss: 0.3170 - val_accuracy: 0.8436 - val_loss: 0.3284 - mcc: 0.6862
Epoch 3/20
183/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8503 - loss: 0.3194
Epoch 3 - MCC: 0.6836
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8503 - loss: 0.3194 - val_accuracy: 0.8419 - val_loss: 0.3306 - mcc: 0.6836
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8491 - loss: 0.3187
Epoch 4 - MCC: 0.6887
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8491 - loss: 0.3187 - val_accuracy: 0.8449 - val_loss: 0.3233 - mcc: 0.6887
Epoch 5/20
185

Training Model: MLP, Fold: 4
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6941 - loss: 0.5159
Epoch 1 - MCC: 0.7024
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6946 - loss: 0.5153 - val_accuracy: 0.8517 - val_loss: 0.3201 - mcc: 0.7024
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8472 - loss: 0.3255
Epoch 2 - MCC: 0.7032
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8472 - loss: 0.3255 - val_accuracy: 0.8519 - val_loss: 0.3163 - mcc: 0.7032
Epoch 3/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8466 - loss: 0.3246
Epoch 3 - MCC: 0.7038
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8467 - loss: 0.3246 - val_accuracy: 0.8521 - val_loss: 0.3157 - mcc: 0.7038
Epoch 4/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8478 - loss: 0.3210
Epoch 4 - MCC: 0.7022
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8478 - loss: 0.3210 - val_accuracy: 0.8516 - val_loss: 0.3152 - mcc: 0.7022
Epoch 5/20
184

Training Model: MLP, Fold: 5
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7246 - loss: 0.5676
Epoch 1 - MCC: 0.6968
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7250 - loss: 0.5670 - val_accuracy: 0.8481 - val_loss: 0.3250 - mcc: 0.6968
Epoch 2/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8478 - loss: 0.3237
Epoch 2 - MCC: 0.6961
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8478 - loss: 0.3237 - val_accuracy: 0.8486 - val_loss: 0.3205 - mcc: 0.6961
Epoch 3/20
179/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8474 - loss: 0.3225
Epoch 3 - MCC: 0.6978
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8475 - loss: 0.3224 - val_accuracy: 0.8492 - val_loss: 0.3195 - mcc: 0.6978
Epoch 4/20
174/188 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8476 - loss: 0.3212
Epoch 4 - MCC: 0.6974
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8476 - loss: 0.3212 - val_accuracy: 0.8480 - val_loss: 0.3233 - mcc: 0.6974
Epoch 5/20
188

{'Accuracy': {'max': np.float64(0.8512853333333333),
              'mean': np.float64(0.8490144000000001),
              'min': np.float64(0.8453173333333334),
              'std': np.float64(0.002176319976474028)},
 'Inference Time (s/sample)': {'max': np.float64(0.0002574790318806966),
                               'mean': np.float64(0.00021616274515787762),
                               'min': np.float64(0.00015666325887044272),
                               'std': np.float64(4.828813564362842e-05)},
 'MCC': {'max': np.float64(0.7034255604036633),
         'mean': np.float64(0.6974780768638038),
         'min': np.float64(0.6896298482187477),
         'std': np.float64(0.004875984816136057)},
 'Parameters': 4713,
 'Train Time (s)': {'max': np.float64(27.538688898086548),
                    'mean': np.float64(26.18506588935852),
                    'min': np.float64(24.72655439376831),
                    'std': np.float64(1.001956715369044)},
 'Training Accuracy': [[0.7957436442

In [ ]:
# All Features Binary

model_results, trained_models = train_and_evaluate(simple_models_dict, X=all_data_vec_binary, y=label_binary, epochs=20, basePath=basePath, dir_name='allF_binary')

filePath = f"{basePath}/AllF_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting


Binary: True
Training Model: LSTM, Fold: 1
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7434 - loss: 0.4606
Epoch 1 - MCC: 0.8189
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7439 - loss: 0.4600 - val_accuracy: 0.9096 - val_loss: 0.2192 - mcc: 0.8189
Epoch 2/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9129 - loss: 0.2120
Epoch 2 - MCC: 0.8397
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9129 - loss: 0.2120 - val_accuracy: 0.9199 - val_loss: 0.1968 - mcc: 0.8397
Epoch 3/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9181 - loss: 0.1983
Epoch 3 - MCC: 0.8444
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9181 - loss: 0.1983 - val_accuracy: 0.9223 - val_loss: 0.1882 - mcc: 0.8444
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9219 - loss: 0.1870
Epoch 4 - MCC: 0.8531
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9219 - loss: 0.1870 - val_accuracy: 0.9267 - val_loss: 0.1790 - mcc: 

Training Model: LSTM, Fold: 2
Epoch 1/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7247 - loss: 0.5077
Epoch 1 - MCC: 0.8164
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7269 - loss: 0.5048 - val_accuracy: 0.9084 - val_loss: 0.2282 - mcc: 0.8164
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9069 - loss: 0.2263
Epoch 2 - MCC: 0.8314
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9069 - loss: 0.2262 - val_accuracy: 0.9155 - val_loss: 0.2066 - mcc: 0.8314
Epoch 3/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9169 - loss: 0.2021
Epoch 3 - MCC: 0.8419
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9169 - loss: 0.2021 - val_accuracy: 0.9211 - val_loss: 0.1907 - mcc: 0.8419
Epoch 4/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9203 - loss: 0.1932
Epoch 4 - MCC: 0.8487
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9204 - loss: 0.1931 - val_accuracy: 0.9246 - val_loss: 0.1834 - mcc: 0.8487
Epoch 

Training Model: LSTM, Fold: 3
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7120 - loss: 0.4766
Epoch 1 - MCC: 0.8212
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7127 - loss: 0.4758 - val_accuracy: 0.9109 - val_loss: 0.2240 - mcc: 0.8212
Epoch 2/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9111 - loss: 0.2166
Epoch 2 - MCC: 0.8312
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9112 - loss: 0.2165 - val_accuracy: 0.9158 - val_loss: 0.2039 - mcc: 0.8312
Epoch 3/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9172 - loss: 0.2008
Epoch 3 - MCC: 0.8402
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9172 - loss: 0.2008 - val_accuracy: 0.9204 - val_loss: 0.1922 - mcc: 0.8402
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9236 - loss: 0.1858
Epoch 4 - MCC: 0.8451
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9235 - loss: 0.1858 - val_accuracy: 0.9229 - val_loss: 0.1886 - mcc: 0.8451
Epoch 

Training Model: LSTM, Fold: 4
Epoch 1/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7191 - loss: 0.4882
Epoch 1 - MCC: 0.8132
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7202 - loss: 0.4869 - val_accuracy: 0.9067 - val_loss: 0.2272 - mcc: 0.8132
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9080 - loss: 0.2250
Epoch 2 - MCC: 0.8313
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9080 - loss: 0.2249 - val_accuracy: 0.9156 - val_loss: 0.2004 - mcc: 0.8313
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9186 - loss: 0.1968
Epoch 3 - MCC: 0.8361
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9186 - loss: 0.1968 - val_accuracy: 0.9181 - val_loss: 0.1971 - mcc: 0.8361
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9193 - loss: 0.1951
Epoch 4 - MCC: 0.8459
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9193 - loss: 0.1950 - val_accuracy: 0.9230 - val_loss: 0.1836 - mcc: 0.8459
Epoch 

Training Model: LSTM, Fold: 5
Epoch 1/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7382 - loss: 0.4863
Epoch 1 - MCC: 0.8190
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7392 - loss: 0.4849 - val_accuracy: 0.9098 - val_loss: 0.2203 - mcc: 0.8190
Epoch 2/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9100 - loss: 0.2171
Epoch 2 - MCC: 0.8278
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9100 - loss: 0.2170 - val_accuracy: 0.9140 - val_loss: 0.2043 - mcc: 0.8278
Epoch 3/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9158 - loss: 0.2011
Epoch 3 - MCC: 0.8420
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9158 - loss: 0.2011 - val_accuracy: 0.9212 - val_loss: 0.1937 - mcc: 0.8420
Epoch 4/20
186/188 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9204 - loss: 0.1923
Epoch 4 - MCC: 0.8461
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9204 - loss: 0.1922 - val_accuracy: 0.9233 - val_loss: 0.1858 - mcc: 0.8461
Epoch 

...

Training Model: MLP, Fold: 2
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7311 - loss: 0.5376
Epoch 1 - MCC: 0.7754
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.7316 - loss: 0.5369 - val_accuracy: 0.8881 - val_loss: 0.2606 - mcc: 0.7754
Epoch 2/20
187/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8851 - loss: 0.2640
Epoch 2 - MCC: 0.7816
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8852 - loss: 0.2639 - val_accuracy: 0.8895 - val_loss: 0.2545 - mcc: 0.7816
Epoch 3/20
181/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8909 - loss: 0.2481
Epoch 3 - MCC: 0.7981
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8910 - loss: 0.2478 - val_accuracy: 0.8991 - val_loss: 0.2299 - mcc: 0.7981
Epoch 4/20
182/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8997 - loss: 0.2303
Epoch 4 - MCC: 0.8142
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8998 - loss: 0.2302 - val_accuracy: 0.9074 - val_loss: 0.2142 - mcc: 0.8142
Epoch 5/20
178

Training Model: MLP, Fold: 3
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6992 - loss: 0.5171
Epoch 1 - MCC: 0.7716
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6998 - loss: 0.5164 - val_accuracy: 0.8862 - val_loss: 0.2640 - mcc: 0.7716
Epoch 2/20
182/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8900 - loss: 0.2521
Epoch 2 - MCC: 0.7850
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8901 - loss: 0.2520 - val_accuracy: 0.8924 - val_loss: 0.2550 - mcc: 0.7850
Epoch 3/20
179/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8937 - loss: 0.2437
Epoch 3 - MCC: 0.7960
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8939 - loss: 0.2433 - val_accuracy: 0.8982 - val_loss: 0.2338 - mcc: 0.7960
Epoch 4/20
184/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9010 - loss: 0.2273
Epoch 4 - MCC: 0.8084
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9011 - loss: 0.2272 - val_accuracy: 0.9045 - val_loss: 0.2216 - mcc: 0.8084
Epoch 5/20
185

Training Model: MLP, Fold: 4
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7130 - loss: 0.5171
Epoch 1 - MCC: 0.7718
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.7135 - loss: 0.5164 - val_accuracy: 0.8855 - val_loss: 0.2594 - mcc: 0.7718
Epoch 2/20
180/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8886 - loss: 0.2558
Epoch 2 - MCC: 0.7959
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8887 - loss: 0.2556 - val_accuracy: 0.8980 - val_loss: 0.2373 - mcc: 0.7959
Epoch 3/20
178/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8984 - loss: 0.2328
Epoch 3 - MCC: 0.8171
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8985 - loss: 0.2326 - val_accuracy: 0.9088 - val_loss: 0.2123 - mcc: 0.8171
Epoch 4/20
183/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9098 - loss: 0.2108
Epoch 4 - MCC: 0.8205
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9098 - loss: 0.2108 - val_accuracy: 0.9101 - val_loss: 0.2083 - mcc: 0.8205
Epoch 5/20
187

Training Model: MLP, Fold: 5
Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7262 - loss: 0.4832
Epoch 1 - MCC: 0.7729
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7268 - loss: 0.4825 - val_accuracy: 0.8868 - val_loss: 0.2594 - mcc: 0.7729
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8924 - loss: 0.2477
Epoch 2 - MCC: 0.7851
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8924 - loss: 0.2477 - val_accuracy: 0.8929 - val_loss: 0.2435 - mcc: 0.7851
Epoch 3/20
183/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8977 - loss: 0.2343
Epoch 3 - MCC: 0.8001
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8977 - loss: 0.2343 - val_accuracy: 0.9004 - val_loss: 0.2287 - mcc: 0.8001
Epoch 4/20
185/188 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9047 - loss: 0.2203
Epoch 4 - MCC: 0.8070
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9047 - loss: 0.2203 - val_accuracy: 0.9025 - val_loss: 0.2255 - mcc: 0.8070
Epoch 5/20
188

{'Accuracy': {'max': np.float64(0.9230506666666667),
              'mean': np.float64(0.9210074666666666),
              'min': np.float64(0.918368),
              'std': np.float64(0.0018559849807821612)},
 'Inference Time (s/sample)': {'max': np.float64(0.00028715960184733075),
                               'mean': np.float64(0.0002304522196451823),
                               'min': np.float64(0.00017362403869628906),
                               'std': np.float64(4.659520243726116e-05)},
 'MCC': {'max': np.float64(0.8463657037253322),
         'mean': np.float64(0.8419326058943973),
         'min': np.float64(0.8361843377902796),
         'std': np.float64(0.0037963567423610826)},
 'Parameters': 5033,
 'Train Time (s)': {'max': np.float64(29.38823437690735),
                    'mean': np.float64(27.78469309806824),
                    'min': np.float64(25.886887311935425),
                    'std': np.float64(1.2216314259244963)},
 'Training Accuracy': [[0.8235800862312317,

### Multi Models

In [ ]:
# Signal Multi

model_results, trained_models = train_and_evaluate(simple_models_dict, X=signal_data_vec_multi, y=label_multi, epochs=20, basePath=basePath, dir_name='signal_multi')

filePath = f"{basePath}/Signal_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

In [ ]:
# Hilbert Multi

model_results, trained_models = train_and_evaluate(simple_models_dict, X=hilbert_data_vec_multi, y=label_multi, epochs=20, basePath=basePath, dir_name='hilbert_multi')

filePath = f"{basePath}/Hilbert_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

In [ ]:
# All Features Multi

model_results, trained_models = train_and_evaluate(simple_models_dict, X=all_data_vec_multi, y=label_multi, epochs=20, batch_size=32, basePath=basePath, dir_name='allF_multi')

filePath = f"{basePath}/AllF_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

Binary: False
Training Model: LSTM, Fold: 1
Epoch 1/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6096 - loss: 1.0006
Epoch 1 - MCC: 0.7467
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.6107 - loss: 0.9981 - val_accuracy: 0.8364 - val_loss: 0.4395 - mcc: 0.7467
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8435 - loss: 0.4336
Epoch 2 - MCC: 0.7781
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8435 - loss: 0.4336 - val_accuracy: 0.8555 - val_loss: 0.3975 - mcc: 0.7781
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8597 - loss: 0.3814
Epoch 3 - MCC: 0.7984
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.8597 - loss: 0.3814 - val_accuracy: 0.8684 - val_loss: 0.3545 - mcc: 0.7984
Epoch 4/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8671 - loss: 0.3587
Epoch 4 - MCC: 0.7842
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.8671 - loss: 0.3588 - val_accuracy: 0.8600 - val_loss: 0.3877 - mc

Training Model: LSTM, Fold: 2
Epoch 1/20
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6647 - loss: 0.9030
Epoch 1 - MCC: 0.7715
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.6652 - loss: 0.9015 - val_accuracy: 0.8516 - val_loss: 0.4129 - mcc: 0.7715
Epoch 2/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8503 - loss: 0.4071
Epoch 2 - MCC: 0.7947
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8504 - loss: 0.4070 - val_accuracy: 0.8670 - val_loss: 0.3590 - mcc: 0.7947
Epoch 3/20
372/375 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8686 - loss: 0.3514
Epoch 3 - MCC: 0.8124
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.8686 - loss: 0.3513 - val_accuracy: 0.8783 - val_loss: 0.3222 - mcc: 0.8124
Epoch 4/20
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8755 - loss: 0.3302
Epoch 4 - MCC: 0.8176
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.8755 - loss: 0.3301 - val_accuracy: 0.8811 - val_loss: 0.3115 - mcc: 0.8176
Epoc

Training Model: LSTM, Fold: 3
Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6675 - loss: 0.8877
Epoch 1 - MCC: 0.7673
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.6678 - loss: 0.8870 - val_accuracy: 0.8485 - val_loss: 0.4138 - mcc: 0.7673
Epoch 2/20
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8517 - loss: 0.4031
Epoch 2 - MCC: 0.7896
375/375 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.8517 - loss: 0.4031 - val_accuracy: 0.8618 - val_loss: 0.3707 - mcc: 0.7896
Epoch 3/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8656 - loss: 0.3593
Epoch 3 - MCC: 0.8014
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.8656 - loss: 0.3592 - val_accuracy: 0.8709 - val_loss: 0.3365 - mcc: 0.8014
Epoch 4/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8739 - loss: 0.3293
Epoch 4 - MCC: 0.8107
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8739 - loss: 0.3293 - val_accuracy: 0.8751 - val_loss: 0.3200 - mcc: 0.8107
Ep

Training Model: LSTM, Fold: 4
Epoch 1/20
372/375 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6502 - loss: 0.9320
Epoch 1 - MCC: 0.7571
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.6514 - loss: 0.9290 - val_accuracy: 0.8440 - val_loss: 0.4262 - mcc: 0.7571
Epoch 2/20
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8521 - loss: 0.4062
Epoch 2 - MCC: 0.7892
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8521 - loss: 0.4061 - val_accuracy: 0.8623 - val_loss: 0.3710 - mcc: 0.7892
Epoch 3/20
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8683 - loss: 0.3508
Epoch 3 - MCC: 0.8050
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8683 - loss: 0.3508 - val_accuracy: 0.8723 - val_loss: 0.3365 - mcc: 0.8050
Epoch 4/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8775 - loss: 0.3219
Epoch 4 - MCC: 0.8187
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8775 - loss: 0.3219 - val_accuracy: 0.8811 - val_loss: 0.3079 - mcc: 0.8187
Ep

Training Model: LSTM, Fold: 5
Epoch 1/20
372/375 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6477 - loss: 0.9196
Epoch 1 - MCC: 0.7563
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.6490 - loss: 0.9166 - val_accuracy: 0.8422 - val_loss: 0.4285 - mcc: 0.7563
Epoch 2/20
372/375 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8500 - loss: 0.4062
Epoch 2 - MCC: 0.7872
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.8500 - loss: 0.4060 - val_accuracy: 0.8607 - val_loss: 0.3723 - mcc: 0.7872
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8670 - loss: 0.3565
Epoch 3 - MCC: 0.8003
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8670 - loss: 0.3565 - val_accuracy: 0.8703 - val_loss: 0.3454 - mcc: 0.8003
Epoch 4/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8744 - loss: 0.3350
Epoch 4 - MCC: 0.8106
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.8744 - loss: 0.3349 - val_accuracy: 0.8767 - val_loss: 0.3238 - mcc: 0.8106
Epo

...

Training Model: MLP, Fold: 2
Epoch 1/20
367/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6813 - loss: 0.9222
Epoch 1 - MCC: 0.7630
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.6836 - loss: 0.9150 - val_accuracy: 0.8471 - val_loss: 0.3982 - mcc: 0.7630
Epoch 2/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8444 - loss: 0.4060
Epoch 2 - MCC: 0.7807
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8444 - loss: 0.4059 - val_accuracy: 0.8582 - val_loss: 0.3669 - mcc: 0.7807
Epoch 3/20
366/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8549 - loss: 0.3768
Epoch 3 - MCC: 0.7869
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8550 - loss: 0.3766 - val_accuracy: 0.8621 - val_loss: 0.3574 - mcc: 0.7869
Epoch 4/20
369/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8594 - loss: 0.3658
Epoch 4 - MCC: 0.7921
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8594 - loss: 0.3657 - val_accuracy: 0.8652 - val_loss: 0.3494 - mcc: 0.7921
Epoch 5/20
362

Training Model: MLP, Fold: 3
Epoch 1/20
372/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6761 - loss: 0.9229
Epoch 1 - MCC: 0.7592
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.6772 - loss: 0.9196 - val_accuracy: 0.8442 - val_loss: 0.4055 - mcc: 0.7592
Epoch 2/20
371/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8480 - loss: 0.3953
Epoch 2 - MCC: 0.7730
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8480 - loss: 0.3952 - val_accuracy: 0.8519 - val_loss: 0.3841 - mcc: 0.7730
Epoch 3/20
370/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8562 - loss: 0.3726
Epoch 3 - MCC: 0.7809
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8562 - loss: 0.3726 - val_accuracy: 0.8580 - val_loss: 0.3677 - mcc: 0.7809
Epoch 4/20
371/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8596 - loss: 0.3624
Epoch 4 - MCC: 0.7911
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8597 - loss: 0.3623 - val_accuracy: 0.8636 - val_loss: 0.3521 - mcc: 0.7911
Epoch 5/20
374

Training Model: MLP, Fold: 4
Epoch 1/20
361/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6869 - loss: 0.8963
Epoch 1 - MCC: 0.7542
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.6908 - loss: 0.8842 - val_accuracy: 0.8424 - val_loss: 0.4088 - mcc: 0.7542
Epoch 2/20
361/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8485 - loss: 0.3910
Epoch 2 - MCC: 0.7738
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8486 - loss: 0.3908 - val_accuracy: 0.8545 - val_loss: 0.3782 - mcc: 0.7738
Epoch 3/20
361/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8572 - loss: 0.3687
Epoch 3 - MCC: 0.7821
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8573 - loss: 0.3686 - val_accuracy: 0.8598 - val_loss: 0.3660 - mcc: 0.7821
Epoch 4/20
367/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8622 - loss: 0.3569
Epoch 4 - MCC: 0.7894
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8623 - loss: 0.3568 - val_accuracy: 0.8626 - val_loss: 0.3536 - mcc: 0.7894
Epoch 5/20
371

Training Model: MLP, Fold: 5
Epoch 1/20
367/375 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6828 - loss: 0.8931
Epoch 1 - MCC: 0.7594
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.6851 - loss: 0.8861 - val_accuracy: 0.8436 - val_loss: 0.4117 - mcc: 0.7594
Epoch 2/20
370/375 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8495 - loss: 0.3915
Epoch 2 - MCC: 0.7688
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8495 - loss: 0.3914 - val_accuracy: 0.8506 - val_loss: 0.3893 - mcc: 0.7688
Epoch 3/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8584 - loss: 0.3678
Epoch 3 - MCC: 0.7815
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8584 - loss: 0.3677 - val_accuracy: 0.8576 - val_loss: 0.3720 - mcc: 0.7815
Epoch 4/20
373/375 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8607 - loss: 0.3598
Epoch 4 - MCC: 0.7879
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8607 - loss: 0.3598 - val_accuracy: 0.8613 - val_loss: 0.3605 - mcc: 0.7879
Epoch 5/20
366

{'Accuracy': {'max': np.float64(0.8808753333333333),
              'mean': np.float64(0.8772389333333332),
              'min': np.float64(0.874086),
              'std': np.float64(0.002662741431098569)},
 'Inference Time (s/sample)': {'max': np.float64(0.00022968300183614094),
                               'mean': np.float64(0.00022425416310628256),
                               'min': np.float64(0.00022078967094421386),
                               'std': np.float64(3.206480511151273e-06)},
 'MCC': {'max': np.float64(0.8159332627699284),
         'mean': np.float64(0.8107765876573618),
         'min': np.float64(0.8059232195126537),
         'std': np.float64(0.0038812405385034296)},
 'Parameters': 6301,
 'Train Time (s)': {'max': np.float64(66.56008172035217),
                    'mean': np.float64(64.39247941970825),
                    'min': np.float64(63.14753985404968),
                    'std': np.float64(1.2117840341047241)},
 'Training Accuracy': [[0.7823675274848938,


# Current SetUp (Deprecated)

In [ ]:
np.unique(label_binary)

In [ ]:
## GOT HERE!!!!

signal_model_results, trained_models = train_and_evaluate(simple_models_dict, X=signal_data_vec, y=label_multi, epochs=10,
                                                           dir_name="signal_multi")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/"
filePath = f"{basePath}/Signal_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(signal_model_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4631 - loss: 1.4171
Epoch 1 - MCC: 0.2942
150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.4644 - loss: 1.4139 - val_accuracy: 0.5936 - val_loss: 1.0309 - mcc: 0.2942
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6458 - loss: 0.9835
Epoch 2 - MCC: 0.5506
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6462 - loss: 0.9827 - val_accuracy: 0.7244 - val_loss: 0.8133 - mcc: 0.5506
Epoch 3/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7194 - loss: 0.8193
Epoch 3 - MCC: 0.5896
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7195 - loss: 0.8188 - val_accuracy: 0.7436 - val_loss: 0.7645 - mcc: 0.5896
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7352 - loss: 0.7733
Epoch 4 - MCC: 0.6064
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.7353 - loss: 0.7732 - val_accuracy: 0.7541 - val_loss: 0.6971 - mcc: 0.6064
Epoch 

Training Model: LSTM, Fold: 2
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5011 - loss: 1.3907
Epoch 1 - MCC: 0.2531
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.5016 - loss: 1.3890 - val_accuracy: 0.5779 - val_loss: 1.0559 - mcc: 0.2531
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6286 - loss: 0.9808
Epoch 2 - MCC: 0.5454
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6291 - loss: 0.9801 - val_accuracy: 0.7209 - val_loss: 0.8381 - mcc: 0.5454
Epoch 3/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7217 - loss: 0.8015
Epoch 3 - MCC: 0.5889
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7219 - loss: 0.8010 - val_accuracy: 0.7427 - val_loss: 0.7309 - mcc: 0.5889
Epoch 4/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7335 - loss: 0.7622
Epoch 4 - MCC: 0.6204
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7336 - loss: 0.7622 - val_accuracy: 0.7642 - val_loss: 0.6856 - mcc: 0.6204
Epoch 

Training Model: LSTM, Fold: 3
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4677 - loss: 1.3863
Epoch 1 - MCC: 0.2289
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.4681 - loss: 1.3854 - val_accuracy: 0.5625 - val_loss: 1.0667 - mcc: 0.2289
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6068 - loss: 1.0210
Epoch 2 - MCC: 0.4761
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.6073 - loss: 1.0202 - val_accuracy: 0.6780 - val_loss: 0.8697 - mcc: 0.4761
Epoch 3/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6978 - loss: 0.8358
Epoch 3 - MCC: 0.5454
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.6979 - loss: 0.8356 - val_accuracy: 0.7188 - val_loss: 0.7709 - mcc: 0.5454
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6990 - loss: 0.8611
Epoch 4 - MCC: 0.5797
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6991 - loss: 0.8608 - val_accuracy: 0.7393 - val_loss: 0.7510 - mcc: 0.5797
Epoch 

Training Model: LSTM, Fold: 4
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4734 - loss: 1.3931
Epoch 1 - MCC: 0.2076
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.4741 - loss: 1.3913 - val_accuracy: 0.5543 - val_loss: 1.1194 - mcc: 0.2076
Epoch 2/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6248 - loss: 1.0125
Epoch 2 - MCC: 0.4649
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.6254 - loss: 1.0111 - val_accuracy: 0.6635 - val_loss: 0.8860 - mcc: 0.4649
Epoch 3/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6902 - loss: 0.8359
Epoch 3 - MCC: 0.5262
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6903 - loss: 0.8360 - val_accuracy: 0.7063 - val_loss: 0.8395 - mcc: 0.5262
Epoch 4/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7227 - loss: 0.7906
Epoch 4 - MCC: 0.5374
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7228 - loss: 0.7904 - val_accuracy: 0.7079 - val_loss: 0.8223 - mcc: 0.5374
Epoch 

Training Model: LSTM, Fold: 5
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4718 - loss: 1.4114
Epoch 1 - MCC: 0.2117
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.4725 - loss: 1.4096 - val_accuracy: 0.5601 - val_loss: 1.1209 - mcc: 0.2117
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5971 - loss: 1.0458
Epoch 2 - MCC: 0.5285
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.5974 - loss: 1.0454 - val_accuracy: 0.7045 - val_loss: 0.8647 - mcc: 0.5285
Epoch 3/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7295 - loss: 0.7867
Epoch 3 - MCC: 0.4541
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.7296 - loss: 0.7865 - val_accuracy: 0.6547 - val_loss: 1.0092 - mcc: 0.4541
Epoch 4/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7059 - loss: 0.8423
Epoch 4 - MCC: 0.6070
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7062 - loss: 0.8411 - val_accuracy: 0.7537 - val_loss: 0.7028 - mcc: 0.6070
Epoch 

...

Training Model: FFN, Fold: 2
Epoch 1/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4936 - loss: 1.4852
Epoch 1 - MCC: 0.1087
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.4946 - loss: 1.4816 - val_accuracy: 0.5462 - val_loss: 1.2185 - mcc: 0.1087
Epoch 2/10
142/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5472 - loss: 1.2303
Epoch 2 - MCC: 0.1923
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5472 - loss: 1.2301 - val_accuracy: 0.5630 - val_loss: 1.2061 - mcc: 0.1923
Epoch 3/10
143/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5582 - loss: 1.2238
Epoch 3 - MCC: 0.2117
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5582 - loss: 1.2235 - val_accuracy: 0.5660 - val_loss: 1.2046 - mcc: 0.2117
Epoch 4/10
144/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5630 - loss: 1.2114
Epoch 4 - MCC: 0.2152
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5629 - loss: 1.2117 - val_accuracy: 0.5660 - val_loss: 1.2025 - mcc: 0.2152
Epoch 5/10
136

Training Model: FFN, Fold: 3
Epoch 1/10
141/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5232 - loss: 1.4409
Epoch 1 - MCC: 0.1965
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5244 - loss: 1.4332 - val_accuracy: 0.5621 - val_loss: 1.2140 - mcc: 0.1965
Epoch 2/10
145/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5608 - loss: 1.2124
Epoch 2 - MCC: 0.2124
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5608 - loss: 1.2125 - val_accuracy: 0.5604 - val_loss: 1.2147 - mcc: 0.2124
Epoch 3/10
141/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5589 - loss: 1.2234
Epoch 3 - MCC: 0.2128
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5591 - loss: 1.2229 - val_accuracy: 0.5602 - val_loss: 1.2152 - mcc: 0.2128
Epoch 4/10
141/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5656 - loss: 1.2082
Epoch 4 - MCC: 0.1996
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5653 - loss: 1.2087 - val_accuracy: 0.5623 - val_loss: 1.2125 - mcc: 0.1996
Epoch 5/10
142

Training Model: FFN, Fold: 4
Epoch 1/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4937 - loss: 1.4661
Epoch 1 - MCC: 0.1792
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.4949 - loss: 1.4625 - val_accuracy: 0.5518 - val_loss: 1.2353 - mcc: 0.1792
Epoch 2/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5622 - loss: 1.2151
Epoch 2 - MCC: 0.1878
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5622 - loss: 1.2151 - val_accuracy: 0.5531 - val_loss: 1.2307 - mcc: 0.1878
Epoch 3/10
144/150 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 1.2149
Epoch 3 - MCC: 0.2023
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5616 - loss: 1.2148 - val_accuracy: 0.5541 - val_loss: 1.2299 - mcc: 0.2023
Epoch 4/10
138/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5640 - loss: 1.2068
Epoch 4 - MCC: 0.1970
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5638 - loss: 1.2073 - val_accuracy: 0.5541 - val_loss: 1.2301 - mcc: 0.1970
Epoch 5/10
141

Training Model: FFN, Fold: 5
Epoch 1/10
145/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5384 - loss: 1.4172
Epoch 1 - MCC: 0.2020
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5387 - loss: 1.4129 - val_accuracy: 0.5578 - val_loss: 1.2249 - mcc: 0.2020
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5614 - loss: 1.2182
Epoch 2 - MCC: 0.2007
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5614 - loss: 1.2181 - val_accuracy: 0.5577 - val_loss: 1.2234 - mcc: 0.2007
Epoch 3/10
137/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5622 - loss: 1.2181
Epoch 3 - MCC: 0.2097
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5620 - loss: 1.2178 - val_accuracy: 0.5576 - val_loss: 1.2237 - mcc: 0.2097
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5599 - loss: 1.2184
Epoch 4 - MCC: 0.2029
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5599 - loss: 1.2183 - val_accuracy: 0.5576 - val_loss: 1.2231 - mcc: 0.2029
Epoch 5/10
147

{'Accuracy': {'max': 0.5659266666666667,
              'mean': 0.5608193333333334,
              'min': 0.5536933333333334,
              'std': 0.004695582037049994},
 'Inference Time (s/sample)': {'max': 0.00035683592160542805,
                               'mean': 0.00024469470977783205,
                               'min': 0.00018439332644144693,
                               'std': 7.209269686743835e-05},
 'MCC': {'max': 0.21045891073996031,
         'mean': 0.2013379324183838,
         'min': 0.19230716278667465,
         'std': 0.0069961739893413585},
 'Parameters': 4117,
 'Train Time (s)': {'max': 15.856249332427979,
                    'mean': 14.264165449142457,
                    'min': 12.252737760543823,
                    'std': 1.3336210124193477},
 'Training Accuracy': [[0.5248016715049744,
                        0.559714138507843,
                        0.5597366690635681,
                        0.5599290728569031,
                        0.5597491264343262,
  

In [ ]:
all_binary_model_results, trained_models = train_and_evaluate(simple_models_dict, X=all_data_vec, y=label_binary, epochs=10,
                                                           dir_name="allF_binary")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/"
filePath = f"{basePath}/AllF_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(all_binary_model_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6777 - loss: 0.5885
Epoch 1 - MCC: 0.6886
150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.6789 - loss: 0.5871 - val_accuracy: 0.8429 - val_loss: 0.3468 - mcc: 0.6886
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8505 - loss: 0.3371
Epoch 2 - MCC: 0.7294
150/150 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.8505 - loss: 0.3370 - val_accuracy: 0.8647 - val_loss: 0.3033 - mcc: 0.7294
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8640 - loss: 0.3033
Epoch 3 - MCC: 0.7519
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.8641 - loss: 0.3033 - val_accuracy: 0.8762 - val_loss: 0.2813 - mcc: 0.7519
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8668 - loss: 0.2955
Epoch 4 - MCC: 0.7655
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8668 - loss: 0.2954 - val_accuracy: 0.8831 - val_loss: 0.2632 - mcc: 0.7655
Epoch 

Training Model: LSTM, Fold: 2
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6915 - loss: 0.5793
Epoch 1 - MCC: 0.6858
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.6926 - loss: 0.5780 - val_accuracy: 0.8433 - val_loss: 0.3518 - mcc: 0.6858
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8471 - loss: 0.3391
Epoch 2 - MCC: 0.7248
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.8472 - loss: 0.3389 - val_accuracy: 0.8625 - val_loss: 0.3049 - mcc: 0.7248
Epoch 3/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8650 - loss: 0.3000
Epoch 3 - MCC: 0.7411
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8651 - loss: 0.2998 - val_accuracy: 0.8711 - val_loss: 0.2838 - mcc: 0.7411
Epoch 4/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8733 - loss: 0.2795
Epoch 4 - MCC: 0.7568
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8733 - loss: 0.2795 - val_accuracy: 0.8787 - val_loss: 0.2699 - mcc: 0.7568
Epoch 

Training Model: LSTM, Fold: 3
Epoch 1/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7055 - loss: 0.5764
Epoch 1 - MCC: 0.6659
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7070 - loss: 0.5743 - val_accuracy: 0.8323 - val_loss: 0.3641 - mcc: 0.6659
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8532 - loss: 0.3306
Epoch 2 - MCC: 0.6986
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8532 - loss: 0.3305 - val_accuracy: 0.8498 - val_loss: 0.3349 - mcc: 0.6986
Epoch 3/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8619 - loss: 0.3055
Epoch 3 - MCC: 0.7277
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.8620 - loss: 0.3053 - val_accuracy: 0.8644 - val_loss: 0.2993 - mcc: 0.7277
Epoch 4/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8725 - loss: 0.2816
Epoch 4 - MCC: 0.7333
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8726 - loss: 0.2815 - val_accuracy: 0.8672 - val_loss: 0.2901 - mcc: 0.7333
Epoch 

Training Model: LSTM, Fold: 4
Epoch 1/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7224 - loss: 0.5651
Epoch 1 - MCC: 0.6693
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7241 - loss: 0.5625 - val_accuracy: 0.8339 - val_loss: 0.3735 - mcc: 0.6693
Epoch 2/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8430 - loss: 0.3489
Epoch 2 - MCC: 0.7119
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.8431 - loss: 0.3488 - val_accuracy: 0.8558 - val_loss: 0.3190 - mcc: 0.7119
Epoch 3/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8609 - loss: 0.3105
Epoch 3 - MCC: 0.7422
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8609 - loss: 0.3105 - val_accuracy: 0.8718 - val_loss: 0.2889 - mcc: 0.7422
Epoch 4/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8688 - loss: 0.2928
Epoch 4 - MCC: 0.7536
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8688 - loss: 0.2927 - val_accuracy: 0.8773 - val_loss: 0.2748 - mcc: 0.7536
Epoch 

Training Model: LSTM, Fold: 5
Epoch 1/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6869 - loss: 0.5805
Epoch 1 - MCC: 0.6819
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.6886 - loss: 0.5783 - val_accuracy: 0.8410 - val_loss: 0.3493 - mcc: 0.6819
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8504 - loss: 0.3320
Epoch 2 - MCC: 0.7212
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8504 - loss: 0.3319 - val_accuracy: 0.8609 - val_loss: 0.3070 - mcc: 0.7212
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8694 - loss: 0.2886
Epoch 3 - MCC: 0.7412
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8694 - loss: 0.2886 - val_accuracy: 0.8707 - val_loss: 0.2874 - mcc: 0.7412
Epoch 4/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8742 - loss: 0.2766
Epoch 4 - MCC: 0.7501
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.8742 - loss: 0.2766 - val_accuracy: 0.8754 - val_loss: 0.2735 - mcc: 0.7501
Epoch 

...

Training Model: FFN, Fold: 2
Epoch 1/10
136/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6996 - loss: 0.5746
Epoch 1 - MCC: 0.7030
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7078 - loss: 0.5642 - val_accuracy: 0.8516 - val_loss: 0.3288 - mcc: 0.7030
Epoch 2/10
136/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8555 - loss: 0.3232
Epoch 2 - MCC: 0.7298
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8558 - loss: 0.3224 - val_accuracy: 0.8654 - val_loss: 0.2993 - mcc: 0.7298
Epoch 3/10
133/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8635 - loss: 0.3046
Epoch 3 - MCC: 0.7341
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8637 - loss: 0.3044 - val_accuracy: 0.8677 - val_loss: 0.2943 - mcc: 0.7341
Epoch 4/10
136/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8657 - loss: 0.3001
Epoch 4 - MCC: 0.7330
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8659 - loss: 0.2997 - val_accuracy: 0.8628 - val_loss: 0.3050 - mcc: 0.7330
Epoch 5/10
133

Training Model: FFN, Fold: 3
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6924 - loss: 0.5763
Epoch 1 - MCC: 0.6866
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6929 - loss: 0.5756 - val_accuracy: 0.8411 - val_loss: 0.3536 - mcc: 0.6866
Epoch 2/10
135/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8526 - loss: 0.3295
Epoch 2 - MCC: 0.7181
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8531 - loss: 0.3284 - val_accuracy: 0.8594 - val_loss: 0.3137 - mcc: 0.7181
Epoch 3/10
133/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8629 - loss: 0.3055
Epoch 3 - MCC: 0.7223
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8632 - loss: 0.3048 - val_accuracy: 0.8618 - val_loss: 0.3069 - mcc: 0.7223
Epoch 4/10
130/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8759 - loss: 0.2779
Epoch 4 - MCC: 0.7249
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8749 - loss: 0.2800 - val_accuracy: 0.8632 - val_loss: 0.3036 - mcc: 0.7249
Epoch 5/10
141

Training Model: FFN, Fold: 4
Epoch 1/10
138/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7082 - loss: 0.5805
Epoch 1 - MCC: 0.6989
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7147 - loss: 0.5713 - val_accuracy: 0.8502 - val_loss: 0.3325 - mcc: 0.6989
Epoch 2/10
138/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8552 - loss: 0.3223
Epoch 2 - MCC: 0.7231
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8553 - loss: 0.3220 - val_accuracy: 0.8624 - val_loss: 0.3082 - mcc: 0.7231
Epoch 3/10
139/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8625 - loss: 0.3047
Epoch 3 - MCC: 0.7347
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8624 - loss: 0.3049 - val_accuracy: 0.8673 - val_loss: 0.2991 - mcc: 0.7347
Epoch 4/10
144/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8679 - loss: 0.2927
Epoch 4 - MCC: 0.7377
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8678 - loss: 0.2930 - val_accuracy: 0.8680 - val_loss: 0.2981 - mcc: 0.7377
Epoch 5/10
143

Training Model: FFN, Fold: 5
Epoch 1/10
135/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6785 - loss: 0.5898
Epoch 1 - MCC: 0.7096
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6888 - loss: 0.5777 - val_accuracy: 0.8547 - val_loss: 0.3241 - mcc: 0.7096
Epoch 2/10
135/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8563 - loss: 0.3217
Epoch 2 - MCC: 0.7280
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8568 - loss: 0.3206 - val_accuracy: 0.8627 - val_loss: 0.3050 - mcc: 0.7280
Epoch 3/10
144/150 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8683 - loss: 0.2942
Epoch 3 - MCC: 0.7300
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8682 - loss: 0.2943 - val_accuracy: 0.8654 - val_loss: 0.2984 - mcc: 0.7300
Epoch 4/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8666 - loss: 0.2960
Epoch 4 - MCC: 0.7299
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8667 - loss: 0.2959 - val_accuracy: 0.8652 - val_loss: 0.2973 - mcc: 0.7299
Epoch 5/10
144

{'Accuracy': {'max': 0.8760366666666667,
              'mean': 0.8735573333333333,
              'min': 0.8700566666666667,
              'std': 0.002543676604182727},
 'Inference Time (s/sample)': {'max': 0.0003952936331431071,
                               'mean': 0.0002770737012227376,
                               'min': 0.00024331251780192057,
                               'std': 5.917718655970826e-05},
 'MCC': {'max': 0.7518154229988196,
         'mean': 0.7476840223246175,
         'min': 0.7391602516522487,
         'std': 0.004653695825672912},
 'Parameters': 5585,
 'Train Time (s)': {'max': 14.646459341049194,
                    'mean': 14.276570510864257,
                    'min': 13.96598219871521,
                    'std': 0.23330843147317679},
 'Training Accuracy': [[0.7730276584625244,
                        0.8553931713104248,
                        0.8622990846633911,
                        0.8648591041564941,
                        0.8666584491729736,
      

In [ ]:
hilbert_binary_model_results, trained_models = train_and_evaluate(simple_models_dict, X=hilbert_data_vec, y=label_binary, epochs=10,
                                                           dir_name="hilbert_binary")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/"
filePath = f"{basePath}/Hilbert_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(hilbert_binary_model_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6560 - loss: 0.6141
Epoch 1 - MCC: 0.6221
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.6570 - loss: 0.6129 - val_accuracy: 0.8080 - val_loss: 0.4002 - mcc: 0.6221
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8243 - loss: 0.3807
Epoch 2 - MCC: 0.7010
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8246 - loss: 0.3802 - val_accuracy: 0.8511 - val_loss: 0.3283 - mcc: 0.7010
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8541 - loss: 0.3218
Epoch 3 - MCC: 0.7200
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.8541 - loss: 0.3218 - val_accuracy: 0.8606 - val_loss: 0.3065 - mcc: 0.7200
Epoch 4/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8567 - loss: 0.3148
Epoch 4 - MCC: 0.7283
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8567 - loss: 0.3147 - val_accuracy: 0.8647 - val_loss: 0.2938 - mcc: 0.7283
Epoch 

Training Model: LSTM, Fold: 2
Epoch 1/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6694 - loss: 0.5914
Epoch 1 - MCC: 0.6436
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6709 - loss: 0.5897 - val_accuracy: 0.8226 - val_loss: 0.3926 - mcc: 0.6436
Epoch 2/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8230 - loss: 0.3834
Epoch 2 - MCC: 0.6890
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8231 - loss: 0.3832 - val_accuracy: 0.8452 - val_loss: 0.3433 - mcc: 0.6890
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8411 - loss: 0.3448
Epoch 3 - MCC: 0.7141
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8412 - loss: 0.3448 - val_accuracy: 0.8563 - val_loss: 0.3190 - mcc: 0.7141
Epoch 4/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8550 - loss: 0.3181
Epoch 4 - MCC: 0.7289
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8551 - loss: 0.3179 - val_accuracy: 0.8647 - val_loss: 0.2962 - mcc: 0.7289
Epoch 

Training Model: LSTM, Fold: 3
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7134 - loss: 0.5896
Epoch 1 - MCC: 0.6280
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.7141 - loss: 0.5884 - val_accuracy: 0.8113 - val_loss: 0.4014 - mcc: 0.6280
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8332 - loss: 0.3624
Epoch 2 - MCC: 0.6893
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.8332 - loss: 0.3623 - val_accuracy: 0.8455 - val_loss: 0.3354 - mcc: 0.6893
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8630 - loss: 0.3052
Epoch 3 - MCC: 0.7142
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8630 - loss: 0.3052 - val_accuracy: 0.8579 - val_loss: 0.3118 - mcc: 0.7142
Epoch 4/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8688 - loss: 0.2919
Epoch 4 - MCC: 0.7231
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8688 - loss: 0.2919 - val_accuracy: 0.8623 - val_loss: 0.3018 - mcc: 0.7231
Epoch 

Training Model: LSTM, Fold: 4
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6954 - loss: 0.5851
Epoch 1 - MCC: 0.6511
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.6958 - loss: 0.5844 - val_accuracy: 0.8229 - val_loss: 0.3812 - mcc: 0.6511
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8350 - loss: 0.3610
Epoch 2 - MCC: 0.6934
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8350 - loss: 0.3608 - val_accuracy: 0.8457 - val_loss: 0.3299 - mcc: 0.6934
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8505 - loss: 0.3241
Epoch 3 - MCC: 0.7140
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8505 - loss: 0.3241 - val_accuracy: 0.8578 - val_loss: 0.3074 - mcc: 0.7140
Epoch 4/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8556 - loss: 0.3134
Epoch 4 - MCC: 0.7301
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8557 - loss: 0.3133 - val_accuracy: 0.8658 - val_loss: 0.2955 - mcc: 0.7301
Epoch 

Training Model: LSTM, Fold: 5
Epoch 1/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6667 - loss: 0.6119
Epoch 1 - MCC: 0.6569
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.6677 - loss: 0.6106 - val_accuracy: 0.8291 - val_loss: 0.3725 - mcc: 0.6569
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8360 - loss: 0.3569
Epoch 2 - MCC: 0.6815
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8360 - loss: 0.3568 - val_accuracy: 0.8395 - val_loss: 0.3463 - mcc: 0.6815
Epoch 3/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8440 - loss: 0.3372
Epoch 3 - MCC: 0.7079
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8441 - loss: 0.3370 - val_accuracy: 0.8541 - val_loss: 0.3168 - mcc: 0.7079
Epoch 4/10
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8552 - loss: 0.3157
Epoch 4 - MCC: 0.7121
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8552 - loss: 0.3156 - val_accuracy: 0.8560 - val_loss: 0.3164 - mcc: 0.7121
Epoch 

...

Training Model: FFN, Fold: 2
Epoch 1/10
138/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6576 - loss: 0.6039
Epoch 1 - MCC: 0.6235
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6646 - loss: 0.5969 - val_accuracy: 0.8084 - val_loss: 0.4081 - mcc: 0.6235
Epoch 2/10
142/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8125 - loss: 0.3959
Epoch 2 - MCC: 0.6405
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8126 - loss: 0.3957 - val_accuracy: 0.8201 - val_loss: 0.3844 - mcc: 0.6405
Epoch 3/10
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8203 - loss: 0.3808
Epoch 3 - MCC: 0.6399
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8203 - loss: 0.3808 - val_accuracy: 0.8208 - val_loss: 0.3799 - mcc: 0.6399
Epoch 4/10
137/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8191 - loss: 0.3822
Epoch 4 - MCC: 0.6422
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8193 - loss: 0.3819 - val_accuracy: 0.8220 - val_loss: 0.3768 - mcc: 0.6422
Epoch 5/10
148

Training Model: FFN, Fold: 3
Epoch 1/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6612 - loss: 0.6262
Epoch 1 - MCC: 0.6045
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6636 - loss: 0.6235 - val_accuracy: 0.8035 - val_loss: 0.4122 - mcc: 0.6045
Epoch 2/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8160 - loss: 0.3945
Epoch 2 - MCC: 0.6229
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8161 - loss: 0.3943 - val_accuracy: 0.8119 - val_loss: 0.3963 - mcc: 0.6229
Epoch 3/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8175 - loss: 0.3888
Epoch 3 - MCC: 0.6000
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8176 - loss: 0.3885 - val_accuracy: 0.7982 - val_loss: 0.4106 - mcc: 0.6000
Epoch 4/10
143/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8210 - loss: 0.3792
Epoch 4 - MCC: 0.6287
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8210 - loss: 0.3791 - val_accuracy: 0.8140 - val_loss: 0.3907 - mcc: 0.6287
Epoch 5/10
147

Training Model: FFN, Fold: 4
Epoch 1/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6320 - loss: 0.6186
Epoch 1 - MCC: 0.6263
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6350 - loss: 0.6160 - val_accuracy: 0.8143 - val_loss: 0.4018 - mcc: 0.6263
Epoch 2/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8095 - loss: 0.4041
Epoch 2 - MCC: 0.6400
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8095 - loss: 0.4040 - val_accuracy: 0.8212 - val_loss: 0.3797 - mcc: 0.6400
Epoch 3/10
141/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8203 - loss: 0.3839
Epoch 3 - MCC: 0.6454
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8201 - loss: 0.3842 - val_accuracy: 0.8234 - val_loss: 0.3755 - mcc: 0.6454
Epoch 4/10
144/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8177 - loss: 0.3866
Epoch 4 - MCC: 0.6448
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8178 - loss: 0.3865 - val_accuracy: 0.8236 - val_loss: 0.3736 - mcc: 0.6448
Epoch 5/10
149

Training Model: FFN, Fold: 5
Epoch 1/10
145/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6235 - loss: 0.6190
Epoch 1 - MCC: 0.6197
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6274 - loss: 0.6160 - val_accuracy: 0.8093 - val_loss: 0.4139 - mcc: 0.6197
Epoch 2/10
146/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8091 - loss: 0.4068
Epoch 2 - MCC: 0.6412
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8093 - loss: 0.4065 - val_accuracy: 0.8191 - val_loss: 0.3876 - mcc: 0.6412
Epoch 3/10
136/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8155 - loss: 0.3913
Epoch 3 - MCC: 0.6430
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8157 - loss: 0.3908 - val_accuracy: 0.8214 - val_loss: 0.3802 - mcc: 0.6430
Epoch 4/10
147/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8192 - loss: 0.3839
Epoch 4 - MCC: 0.6451
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8193 - loss: 0.3838 - val_accuracy: 0.8224 - val_loss: 0.3769 - mcc: 0.6451
Epoch 5/10
144

{'Accuracy': {'max': 0.8281466666666667,
              'mean': 0.822786,
              'min': 0.8105766666666666,
              'std': 0.0062684371257914285},
 'Inference Time (s/sample)': {'max': 0.0003861121336619059,
                               'mean': 0.0003136016527811686,
                               'min': 0.0001863551139831543,
                               'std': 7.591929779999738e-05},
 'MCC': {'max': 0.6560390143039783,
         'mean': 0.6478863229405225,
         'min': 0.630088646598591,
         'std': 0.0093476145518534},
 'Parameters': 4305,
 'Train Time (s)': {'max': 16.060234546661377,
                    'mean': 14.167105054855346,
                    'min': 12.807628870010376,
                    'std': 1.1062535996150167},
 'Training Accuracy': [[0.7417033910751343,
                        0.8108367323875427,
                        0.8162415623664856,
                        0.8173166513442993,
                        0.8187927007675171,
                   

## k-fold with Noise Level Performance Data

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, accuracy_score
from collections import defaultdict

# Assume we have a noise label array (same length as X)
# Example: noise_labels = np.random.randint(0, 3, len(X))  # Simulating 3 noise levels: 0, 1, 2
noise_labels = ...  # This should be an array of shape (len(X),)

# Create directory for model storage
os.makedirs("saved_models", exist_ok=True)

# Define model architectures
model_dict = {
    "Simple_LSTM": lambda input_shape, num_classes: tf.keras.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]),
}

# Number of folds
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize results storage
model_results = {model_name: {} for model_name in model_dict.keys()}
trained_models = {model_name: [] for model_name in model_dict.keys()}

# Convert y to a format compatible with StratifiedKFold
y_flat = np.argmax(y, axis=-1) if y.ndim == 3 else y  # Ensure shape (samples,)

for model_name, model_fn in model_dict.items():
    acc_scores, mcc_scores, train_times, infer_times = [], [], [], []
    noise_performance = defaultdict(lambda: {"acc": [], "mcc": []})  # Store per noise level

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_flat[:, 0])):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        val_noise_levels = noise_labels[val_idx]  # Get noise levels for validation set

        # Build and compile model
        model = model_fn(input_shape=X.shape[1:], num_classes=5)

        # Train model and measure time
        start_train = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val), verbose=1)
        train_time = time.time() - start_train

        # Inference speed measurement
        start_infer = time.time()
        y_pred_probs = model.predict(X_val)
        infer_time = (time.time() - start_infer) / len(X_val)  # Time per sample

        # Compute metrics
        y_pred = np.argmax(y_pred_probs, axis=-1).flatten()
        y_true = y_val.flatten()

        acc = accuracy_score(y_true, y_pred)
        mcc = matthews_corrcoef(y_true, y_pred)

        acc_scores.append(acc)
        mcc_scores.append(mcc)
        train_times.append(train_time)
        infer_times.append(infer_time)

        # Save model for this fold
        model_save_path = f"saved_models/{model_name}_fold_{fold+1}.h5"
        model.save(model_save_path)
        trained_models[model_name].append(model_save_path)

        # **Noise-Level Performance Tracking**
        for noise_level in np.unique(val_noise_levels):
            mask = val_noise_levels == noise_level  # Get indices of this noise level
            y_true_noise = y_true[mask]
            y_pred_noise = y_pred[mask]

            if len(y_true_noise) > 0:  # Avoid empty cases
                acc_noise = accuracy_score(y_true_noise, y_pred_noise)
                mcc_noise = matthews_corrcoef(y_true_noise, y_pred_noise)

                noise_performance[noise_level]["acc"].append(acc_noise)
                noise_performance[noise_level]["mcc"].append(mcc_noise)

    # Store aggregated results
    model_results[model_name] = {
        "Accuracy": {"mean": np.mean(acc_scores), "std": np.std(acc_scores), "min": np.min(acc_scores), "max": np.max(acc_scores)},
        "MCC": {"mean": np.mean(mcc_scores), "std": np.std(mcc_scores), "min": np.min(mcc_scores), "max": np.max(mcc_scores)},
        "Train Time (s)": {"mean": np.mean(train_times), "std": np.std(train_times), "min": np.min(train_times), "max": np.max(train_times)},
        "Inference Time (s/sample)": {"mean": np.mean(infer_times), "std": np.std(infer_times), "min": np.min(infer_times), "max": np.max(infer_times)},
        "Noise Performance": {
            noise_level: {
                "Accuracy": {"mean": np.mean(scores["acc"]), "std": np.std(scores["acc"])},
                "MCC": {"mean": np.mean(scores["mcc"]), "std": np.std(scores["mcc"])}
            } for noise_level, scores in noise_performance.items()
        }
    }

    # Save the final trained model from the last fold
    final_model_save_path = f"saved_models/{model_name}_final.h5"
    model.save(final_model_save_path)

# Print results
import pprint
pprint.pprint(model_results)

# Save results as JSON
import json
with open("model_results.json", "w") as f:
    json.dump(model_results, f, indent=4)


## Visualize Metrics

In [ ]:
def plot_confusion_matrix(cm, class_names, title="Confusion Matrix"):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [ ]:
def plot_confusion_matrix(cm, class_names, title="Confusion Matrix"):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [ ]:
# Loss curve
def plot_loss_curves(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curves')
    plt.show()


## Result Report

In [ ]:
results = {
    "Model": [],
    "Accuracy": [],
    "MAE": [],
    "F1-Score": [],
    "Training Time (s)": [],
    "Inference Time (s)": []
}

# Assuming `models` is a list of your models
for model_name, model in models.items():
    metrics = evaluate_model(model, X_train, y_train, X_val, y_val)
    inference_time = measure_inference_time(model, X_test)

    results["Model"].append(model_name)
    results["Accuracy"].append(metrics["accuracy"])
    results["MAE"].append(metrics["mae"])
    results["F1-Score"].append(metrics["f1"])
    results["Training Time (s)"].append(metrics["training_time"])
    results["Inference Time (s)"].append(inference_time)

# Convert to DataFrame for better visualization
results_df = pd.DataFrame(results)
print(results_df)
